In [1]:
import os
os.chdir('C:\\Users\\KNAGENDRA\\Attrition_modelling')

In [2]:
# load required libraries 
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import configparser
import calendar
import datetime as dt
import bisect
import xlsxwriter
import warnings
import sys
import os

# suppress the warnings
warnings.filterwarnings("ignore")

In [3]:
# create a log file
log= open("Attrition Analytics Log.txt","a")
log.write("********** Opened the log file on  ***********\n")
orig_date= datetime.now()
date_str = datetime.strftime(orig_date, '%Y-%m-%d %H:%M:%S')
log.write(date_str)
log.write("\n")
log.write("Executing Attrition Analytics ETL\r\n")

35

In [4]:
# # get the quarter start dates
# today = dt.date.today()
# qbegins = []
# for yr in [today.year-1, today.year]:
#     if yr == today.year-1:
#         qbegins = [dt.date(yr, month, 1) for month in (4, 7, 10)]
#         qbegins1 = qbegins
#     else:
#         qbegins = [dt.date(yr, month, 1) for month in (1, 4, 7, 10)]
#         qbegins = qbegins + qbegins1
        
# idx = bisect.bisect(qbegins, today)
# qbegins = [qbegins[idx-1], qbegins[idx-2], qbegins[idx-3], qbegins[idx-4]]
# qbegins = pd.to_datetime(qbegins)

# qbegins

# # get the quarter end dates
# qends = []
# for yr in [today.year-1, today.year]:
#     if yr == today.year-1:
#         qends = [dt.date(yr, month, 30) for month in (6, 9, 12)]
#         qends1 = qends
#     else:
#         qends = [dt.date(yr, month, 30) for month in (3, 6, 9, 12)]
#         qends = qends + qends1
        
# idx = bisect.bisect(qends, today)
# qends = [qends[idx],qends[idx-1], qends[idx-2], qends[idx-3]]
# qends = pd.to_datetime(qends)
# qends

# # logic to handle 31st day in a month
# qends1 = []
# for i in range(len(qends)):
#     if qends[i].month in (3, 12):
#         qends1.append((qends[i] + timedelta(days=1)))
#     else:
#         qends1.append(qends[i]) 
# qends1 = pd.to_datetime(qends1)

# #defining year quarter 
# yr_qtr1  = str(qbegins.to_period('Q')[0])
# yr_qtr2  = str(qbegins.to_period('Q')[1])
# yr_qtr3  = str(qbegins.to_period('Q')[2])
# yr_qtr4  = str(qbegins.to_period('Q')[3])

# # converting to datetime
# qbegins = qbegins.to_pydatetime()
# qends1 = qends1.to_pydatetime()

In [5]:
# try:
#     # get the folder name
#     folder = datetime.now().strftime('%b %Y')

#     # define quarter start and end dates
#     q1_start = qbegins[0]
#     q1_end = qends1[0]
    
#     q2_start = qbegins[1]
#     q2_end = qends1[1]

#     q3_start = qbegins[2]
#     q3_end = qends1[2]
    
#     q4_start = qbegins[3]
#     q4_end = qends1[3]

#     print("Successfully defined quarters\r\n")
#     log.write("Successfully defined quarters\r\n")
    
# except:
#     print("Failed to define quarters\r\n")
#     log.write("Failed to define quarters\r\n")
#     log.close()
#     sys.exit(1)

In [6]:
# # get the folder name
# folder = datetime.now().strftime('%b %Y')

# q1_start = pd.datetime(2020,7,1)
# q1_end = pd.datetime(2020,9,30)

# q2_start = pd.datetime(2020,4,1)
# q2_end = pd.datetime(2020,6,30)

# q3_start = pd.datetime(2020,1,1)
# q3_end = pd.datetime(2020,3,31)

# q4_start = pd.datetime(2019,10,1)
# q4_end = pd.datetime(2019,12,31)

# yr_qtr1  = '2020Q3'
# yr_qtr2  = '2020Q2'
# yr_qtr3  = '2020Q1'
# yr_qtr4  = '2019Q4'

In [7]:

# read config file
config = configparser.ConfigParser()
config.read('config_parser.ini')

# folder name
folder = config['folder']['folder']

# define quarter start and end dates
yr_qtr1  = config['quarter definition']['yr_qtr1']
q1_start = pd.datetime(int(config['quarter definition']['y_q1_start']), int(config['quarter definition']['m_q1_start']), int(config['quarter definition']['d_q1_start']))
q1_end = pd.datetime(int(config['quarter definition']['y_q1_end']), int(config['quarter definition']['m_q1_end']), int(config['quarter definition']['d_q1_end']))

yr_qtr2  = config['quarter definition']['yr_qtr2']
q2_start = pd.datetime(int(config['quarter definition']['y_q2_start']), int(config['quarter definition']['m_q2_start']), int(config['quarter definition']['d_q2_start']))
q2_end = pd.datetime(int(config['quarter definition']['y_q2_end']), int(config['quarter definition']['m_q2_end']), int(config['quarter definition']['d_q2_end']))

yr_qtr3  = config['quarter definition']['yr_qtr3']
q3_start = pd.datetime(int(config['quarter definition']['y_q3_start']), int(config['quarter definition']['m_q3_start']), int(config['quarter definition']['d_q3_start']))
q3_end = pd.datetime(int(config['quarter definition']['y_q3_end']), int(config['quarter definition']['m_q3_end']), int(config['quarter definition']['d_q3_end']))

yr_qtr4  = config['quarter definition']['yr_qtr4']
q4_start = pd.datetime(int(config['quarter definition']['y_q4_start']), int(config['quarter definition']['m_q4_start']), int(config['quarter definition']['d_q4_start']))
q4_end = pd.datetime(int(config['quarter definition']['y_q4_end']), int(config['quarter definition']['m_q4_end']), int(config['quarter definition']['d_q4_end']))

print("Successfully read Config file\r\n")


Successfully read Config file



In [8]:
print(yr_qtr1)
print(yr_qtr4)

2020Q3
2019Q4


In [9]:
# function that takes date range(e.g. trip start date and trip end date) and assigns the days into respective quarters
def quarter_days_allocator(data, cols, q_start, q_end):
    return (np.where(
                      ((data[cols[0]] < q_start) &  (data[cols[1]] > q_end)), 
                            data[cols[2]],
                                np.where(
                                        ((data[cols[0]] < q_start) & ((data[cols[1]]  >= q_start) & (data[cols[1]]  <= q_end))),
                                                data[cols[3]],
                                                    np.where(
                                                            (((data[cols[0]] >= q_start) & (data[cols[0]] <= q_end)) & (data[cols[1]] > q_end)),
                                                                    data[cols[4]],
                                                                            np.where(
                                                                                    (((data[cols[0]] >= q_start) & (data[cols[0]] <= q_end)) & ((data[cols[1]] >= q_start) & (data[cols[1]] <= q_end))),
                                                                                            data[cols[5]],
                                                                                                    0)))))


In [10]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Employee Comprehensive Report\\')

In [11]:

# importing employee data and filtering the required data
employee_data = pd.read_excel('Employee Data Comprehensive Report.xlsx', sheet_name = "Sheet1")


employee_data = employee_data[(employee_data['Masked Grade'] != "G012") & 
                          (employee_data['Masked Grade'] != "G013") &
                          (employee_data['Masked Grade'] != "G014") &
                          (employee_data['Masked Grade'] != "G017") &
                          (employee_data['Masked Grade'] != "G007")]

employee_data = employee_data[['Masked emp code.'
                           ,'DATE_OF_BIRTH'
                           ,'Masked gender'
                           ,'MARITAL_STATUS'
                           ,'WEDDING_DATE'
                           ,'Masked Location'
                           ,'PERMNANT_STATE'
                           ,'Masked Entity'
                           ,'Masked emp status'
                           ,'Masked SL'
                           ,'Masked Function'
                           ,'Masked Grade'
                           ,'NOTICE_PERIOD_IN_DAYS'
                           ,'DATE_OF_JOINING'
                           ,'DATE_OF_LEAVING'
                           ,'Masked RM Ids'
                           ,'PARTNER_NAME'
                           ,'REPORTING_TO'
                           ,'EMPLOYEE_STATUS']]

# rename the columns
employee_data = employee_data.rename(columns={"Masked emp code.": "Masked Code",
                                        "Masked gender": "Masked Gender",
                                        "Masked emp status": "Masked Status",
                                        "Masked SL": "Masked Department",
                                        "Masked RM Ids": "Masked RM ID"})

employee_data['DATE_OF_BIRTH'] = pd.to_datetime(employee_data['DATE_OF_BIRTH'])
employee_data['WEDDING_DATE'] = pd.to_datetime(employee_data['WEDDING_DATE'])
employee_data['DATE_OF_JOINING'] = pd.to_datetime(employee_data['DATE_OF_JOINING'])
employee_data['DATE_OF_LEAVING'] = pd.to_datetime(employee_data['DATE_OF_LEAVING'])

# calculate age
employee_data['AGE'] = q1_end - employee_data['DATE_OF_BIRTH']
employee_data['AGE'] = employee_data['AGE']/np.timedelta64(1,'Y')

# calculate years since marriage
employee_data['YEARS_SINCE_MARRIAGE'] = q1_end - employee_data['WEDDING_DATE']
employee_data['YEARS_SINCE_MARRIAGE'] = employee_data['YEARS_SINCE_MARRIAGE']/np.timedelta64(1,'Y')

# calculate tenure at Deloitte
employee_data['YEARS_AT_COMPANY'] = q1_end - employee_data['DATE_OF_JOINING']
employee_data['YEARS_AT_COMPANY'] = employee_data['YEARS_AT_COMPANY']/np.timedelta64(1,'Y')
    
employee_data.head(10)

,Masked Code,DATE_OF_BIRTH,Masked Gender,MARITAL_STATUS,WEDDING_DATE,Masked Location,PERMNANT_STATE,Masked Entity,Masked Status,Masked Department,...,NOTICE_PERIOD_IN_DAYS,DATE_OF_JOINING,DATE_OF_LEAVING,Masked RM ID,PARTNER_NAME,REPORTING_TO,EMPLOYEE_STATUS,AGE,YEARS_SINCE_MARRIAGE,YEARS_AT_COMPANY
0,91EShF/nnLUqp3e+hCL9ag==,1988-12-10,S002,Single,NaT,L014,Maharashtra,E011,ES001,SL083,...,30,2017-01-16,2017-04-21,//u+WyHm0RVCzJMGRkQAJg==,Jaskiran Bhatia,Mallarswami R Nonvinkere,INACTIVE,31.806266,NaN,3.704388
1,fTVAyZmScwfarHFnb7uxcg==,1989-07-26,S002,Single,NaT,L004,NaN,E011,ES001,SL083,...,30,2017-03-20,2017-05-26,//u+WyHm0RVCzJMGRkQAJg==,Jaskiran Bhatia,Mallarswami R Nonvinkere,INACTIVE,31.182023,NaN,3.531900
2,1qJ7mSYktnFiPm11FXL6+Q==,1991-04-01,S001,Married,2016-04-25,L014,Maharashtra,E011,ES001,SL083,...,30,2017-02-06,2017-09-08,//u+WyHm0RVCzJMGRkQAJg==,Jaskiran Bhatia,Mallarswami R Nonvinkere,INACTIVE,29.500948,4.432671,3.646892
3,/Zs6nmpo53o6spqg1984dg==,1983-07-01,S001,Married,2013-05-29,L007,Haryana,E011,ES002,SL173,...,60,2010-01-04,NaT,/+mkiC7vIkQFVDBYg0ER5Q==,Kedar Anil Sawale,Johar Ali Batterywala,ACTIVE,37.251963,7.340329,10.738071
4,j4u0ELqrgMb62Vmncb5QwQ==,1975-07-12,S001,Married,2004-12-26,L014,Maharashtra,E011,ES002,SL173,...,60,2010-04-22,NaT,/+mkiC7vIkQFVDBYg0ER5Q==,Sachin Chandrakant Rasam,Johar Ali Batterywala,ACTIVE,45.222010,15.762131,10.442377
5,rS2aNvJKd9r3uBZQFp5nUA==,1981-03-22,S002,Single,NaT,L014,Maharashtra,E011,ES002,SL173,...,60,2006-02-13,NaT,/+mkiC7vIkQFVDBYg0ER5Q==,Senthilvel Kaliyamurthy,Johar Ali Batterywala,ACTIVE,39.527163,NaN,14.628637
6,OuiPsJYG58cScUbQG7IELA==,1989-06-26,S001,Married,2018-04-05,L004,Tamil nadu,E011,ES002,SL002,...,60,2016-06-13,2019-11-29,/+mkiC7vIkQFVDBYg0ER5Q==,Deepa Seshadri,Johar Ali Batterywala,INACTIVE,31.264160,2.488757,4.298514
7,8KXH9PWiEIntEtXZfxXl2Q==,1988-08-25,S002,Single,NaT,L004,NaN,E011,ES002,SL145,...,60,2014-11-10,2015-11-04,/+mkiC7vIkQFVDBYg0ER5Q==,Sachin Narayan Paranjape,Johar Ali Batterywala,INACTIVE,32.099222,NaN,5.889238
8,S/WXSWtz/WjMJiJu30dNdg==,1984-10-04,S001,Single,NaT,L004,NaN,E013,ES002,SL153,...,60,2010-04-05,2011-08-17,/+mkiC7vIkQFVDBYg0ER5Q==,Santosh Kumar,Johar Ali Batterywala,INACTIVE,35.989788,NaN,10.488922
9,nyvMhOhbC5KRL+pr6qa35A==,1984-04-01,S001,Married,2012-02-12,L004,Karnataka,E011,ES002,SL002,...,60,2015-01-28,2019-11-11,/+mkiC7vIkQFVDBYg0ER5Q==,Johar Ali Batterywala,Johar Ali Batterywala,INACTIVE,36.499038,8.632621,5.672943


In [12]:
len(employee_data['Masked Code'])

27960

In [13]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code/Code/Mapping Table/')

In [14]:

# import the location mapping table and map it with the employee data to get the actual office location
location_mapping = pd.read_excel('Office Location Code Mapping.xlsx', sheet_name = "Sheet1")


employee_data = pd.merge(employee_data, location_mapping, on='Masked Location', how='left')

# IsSameState 
employee_data['IsSameState'] = np.where(employee_data['Office State'] == employee_data['PERMNANT_STATE'], 1, 0)

print("Successfully mapped Office location\r\n")



Successfully mapped Office location



In [15]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Pay Group/')

In [16]:

# importing paygroup data to eliminate non-permanent employees
paygroup_data = pd.read_excel('Pay Group.xlsx', sheet_name = "Sheet1")

paygroup_data['NonPermanent_flag'] = 1
paygroup_data.columns = ['Masked Code','NonPermanent_flag']
employee_data = pd.merge(employee_data, paygroup_data, on='Masked Code', how='left')
employee_data = employee_data[employee_data['NonPermanent_flag'] != 1]

del employee_data['NonPermanent_flag']

print("Successfully executed Pay Group file\r\n")


Successfully executed Pay Group file



In [17]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Resignation Status Report\\')

In [18]:

resignation_data = pd.read_excel('Resignation Status Report.xlsx', sheet_name = "Sheet1") 


resignation_data = resignation_data[resignation_data['SEPARATIONTYPE']== "RESIGNATION"]
resignation_data = resignation_data[resignation_data['STATUS']== "Approved"]
resignation_data = resignation_data[['Masked emp id.','RESIGNATIONLETTERDATE']]
resignation_data.columns = ['Masked Code', 'Resignation Date']

resignation_data['Resignation Date'] = pd.to_datetime(resignation_data['Resignation Date'])

employee_data = pd.merge(employee_data, resignation_data, on='Masked Code', how='left')

employee_data['SeparationFlag']  = np.where(((employee_data['Resignation Date'].isnull()) & (employee_data['EMPLOYEE_STATUS'] == 'INACTIVE')), 1, 0)

employee_data = employee_data[employee_data['SeparationFlag'] == 0]
del employee_data['SeparationFlag']

print("Successfully executed Resignation data\r\n")




Successfully executed Resignation data



In [19]:
# len(employee_data.loc[employee_data['Attrition']==0])

In [20]:
# len(employee_data['Resignation Date'].isnull())

In [21]:

DATA = employee_data
#DATA = employee_data[employee_data['EMPLOYEE_STATUS']== 'ACTIVE']

# creating Class label for Attrition
# DATA['Attrition'] = np.where((DATA['Resignation Date'] <= q1_end), 1, 0)
DATA['Attrition'] = np.where(((DATA['Resignation Date'] >= q1_start) & (DATA['Resignation Date'] <= q1_end)), 1, 0)
DATA['']
# keep only active employees
# DATA = DATA[DATA['Attrition']==0]

print("Successfully defined active employees in considered Quarter\r\n")


Successfully defined active employees in considered Quarter



In [22]:
len(DATA.loc[DATA['Attrition'] == 0])

10353

In [23]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Performance Grouping Data\\')

In [24]:

performance_data = pd.read_excel('Performance Grouping Data.xlsx', sheet_name = "Sheet1")



performance_data.columns = ['Encrypted Emp ID', 'Performance Group']

# check if we have multiple records
check_performance_data = performance_data.groupby(['Encrypted Emp ID'], as_index=False).count()
check_performance_data = check_performance_data.sort_values(by=['Performance Group'], ascending=False)

if ((check_performance_data.iloc[0, 1]) >1):
    print("Data Quality Issue - Performance data has duplicate values")


# enrich with performance data
# performance rating of employees
DATA = pd.merge(DATA, performance_data, left_on='Masked Code', right_on='Encrypted Emp ID', how='left')
DATA = DATA.rename(columns={'Performance Group':'EmpPerformanceRating'}) 

# performance rating of RM's
DATA = pd.merge(DATA, performance_data, left_on='Masked RM ID', right_on='Encrypted Emp ID', how='left')
DATA = DATA.rename(columns={'Performance Group':'RMPerformanceRating'})

print("Successfully executed Performance Grouping Data\r\n")


Successfully executed Performance Grouping Data



In [25]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Career Progression Report\\')

In [26]:

promotion_data = pd.read_excel("Career Progression Report.xlsx", sheet_name = "Sheet1")



# remove only designation changes
promotion_data = promotion_data[promotion_data['Re-designation'].isnull()]


# subset the data to have the required structure
promotion_data_G007 = promotion_data[['Masked Emp Id', 'G007 since', 'DOJ']]
promotion_data_G007.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G007['Grade'] = 'G007'

promotion_data_G003 = promotion_data[['Masked Emp Id', 'G003 Since', 'DOJ']]
promotion_data_G003.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G003['Grade'] = 'G003'

promotion_data_G009 = promotion_data[['Masked Emp Id', 'G009 Since', 'DOJ']]
promotion_data_G009.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G009['Grade'] = 'G009'

promotion_data_G006 = promotion_data[['Masked Emp Id', 'G006 Since.2', 'DOJ']]
promotion_data_G006.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G006['Grade'] = 'G006'

promotion_data_G004 = promotion_data[['Masked Emp Id', 'G004 Since', 'DOJ']]
promotion_data_G004.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G004['Grade'] = 'G004'

promotion_data_G001 = promotion_data[['Masked Emp Id', 'G001 Since', 'DOJ']]
promotion_data_G001.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G001['Grade'] = 'G001'

promotion_data_G008 = promotion_data[['Masked Emp Id', 'G008 Since', 'DOJ']]
promotion_data_G008.columns = ['Masked Code', 'EffectiveDate', 'DOJ']
promotion_data_G008['Grade'] = 'G008'


# concat the datasets
promotion_data = pd.concat([promotion_data_G007, promotion_data_G003, promotion_data_G009, promotion_data_G006,
                            promotion_data_G004, promotion_data_G001, promotion_data_G008], axis=0)



# replace DOJ string with the date of joining
promotion_data['EffectiveDate'] = np.where((promotion_data['EffectiveDate'] == 'Since DOJ'), promotion_data['DOJ'], promotion_data['EffectiveDate'])

del promotion_data['DOJ']

# format the date
promotion_data['EffectiveDate'] = pd.to_datetime(promotion_data['EffectiveDate'], errors='coerce').dt.date
promotion_data['EffectiveDate'] = pd.to_datetime(promotion_data['EffectiveDate'])

# drop the rows with null values
promotion_data = promotion_data.dropna()

# sort the data
promotion_data = promotion_data.sort_values(by=['Masked Code', 'EffectiveDate'], ascending=False)

promotion_data['IsPromoted'] = 'Promoted'

# enrich with promotion data
promotion_data = promotion_data[promotion_data['EffectiveDate'] <= q1_end]
promotion_data = promotion_data.drop_duplicates(subset = 'Masked Code', keep = 'first')

#flag for current and previous quarter promotion
promotion_data['IsPromoted_LatestQtr'] = np.where((promotion_data['EffectiveDate'] >= q1_start) ,1, 0)
promotion_data['IsPromoted_PreviousQtr'] = np.where(((promotion_data['EffectiveDate'] >= q2_start) & (promotion_data['EffectiveDate'] <= q2_end)) ,1, 0)                                                      

DATA = pd.merge(DATA, promotion_data, on='Masked Code', how='left')

print("Successfully executed Career Progression Report\r\n")


Successfully executed Career Progression Report



In [27]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Staff Utilization Report\\')

In [28]:

# import the quarterly utilization data
util_data_Q5 = pd.read_excel('Staff Utilization Report_Q5.xlsx', sheet_name = "Sheet1")
util_data_Q5 = util_data_Q5[['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours']]
util_data_Q5['Util_Flag'] = "SUR_Q5"
util_data_Q5.columns = ['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours', 'Util_Flag']

util_data_Q4 = pd.read_excel('Staff Utilization Report_Q4.xlsx', sheet_name = "Sheet1")
util_data_Q4 = util_data_Q4[['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours']]
util_data_Q4['Util_Flag'] = "SUR_Q4"
util_data_Q4.columns = ['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours', 'Util_Flag']

util_data_Q3 = pd.read_excel('Staff Utilization Report_Q3.xlsx', sheet_name = "Sheet1")
util_data_Q3 = util_data_Q3[['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours']]
util_data_Q3['Util_Flag'] = "SUR_Q3"
util_data_Q3.columns = ['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours', 'Util_Flag']

util_data_Q2 = pd.read_excel('Staff Utilization Report_Q2.xlsx', sheet_name = "Sheet1")
util_data_Q2 = util_data_Q2[['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours']]
util_data_Q2['Util_Flag'] = "SUR_Q2"
util_data_Q2.columns = ['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours', 'Util_Flag']

util_data_Q1 = pd.read_excel('Staff Utilization Report_Q1.xlsx', sheet_name = "Sheet1")
util_data_Q1 = util_data_Q1[['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours']]
util_data_Q1['Util_Flag'] = "SUR_Q1"
util_data_Q1.columns = ['Masked Code', 'Standard Hours', 'Billable Hours', 'Productive Non-Billable Hours', 'Non-Productive Non-Billable Hours', 'Util_Flag']


# concate all the quarterly utilization datasets
staff_util_data = pd.concat([util_data_Q5
                        ,util_data_Q4
                        ,util_data_Q3
                        ,util_data_Q2
                        ,util_data_Q1], axis=0)


# aggregate at employee and quarter level
staff_util_data = staff_util_data.groupby(['Masked Code', 'Util_Flag'], as_index=False).agg({'Standard Hours':'sum'
                                                                                          ,'Billable Hours':'sum'
                                                                                          ,'Productive Non-Billable Hours':'sum'
                                                                                          ,'Non-Productive Non-Billable Hours':'sum'})



# enrich with utilization data
# by lastest year
staff_util_data_latestYr = staff_util_data[(staff_util_data['Util_Flag'] == "SUR_Q1")|
                                           (staff_util_data['Util_Flag'] == "SUR_Q2")| 
                                           (staff_util_data['Util_Flag'] == "SUR_Q3")|
                                           (staff_util_data['Util_Flag'] == "SUR_Q4")|
                                           (staff_util_data['Util_Flag'] == "SUR_Q5")]


# formulae to adjust for quarter definition
staff_util_data_latestYr['Standard Hours'] = np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q1"),
                                                    (staff_util_data_latestYr['Standard Hours'] * 0.33),
                                                         np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q5"),
                                                                  (staff_util_data_latestYr['Standard Hours'] * 0.66),
                                                                      staff_util_data_latestYr['Standard Hours']
                                                ))

staff_util_data_latestYr['Billable Hours'] = np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q1"),
                                                    (staff_util_data_latestYr['Billable Hours'] * 0.33),
                                                         np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q5"),
                                                                  (staff_util_data_latestYr['Billable Hours'] * 0.66),
                                                                      staff_util_data_latestYr['Billable Hours']
                                                ))

staff_util_data_latestYr['Productive Non-Billable Hours'] = np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q1"),
                                                    (staff_util_data_latestYr['Productive Non-Billable Hours'] * 0.33),
                                                         np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q5"),
                                                                  (staff_util_data_latestYr['Productive Non-Billable Hours'] * 0.66),
                                                                      staff_util_data_latestYr['Productive Non-Billable Hours']
                                                ))

staff_util_data_latestYr['Non-Productive Non-Billable Hours'] = np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q1"),
                                                    (staff_util_data_latestYr['Non-Productive Non-Billable Hours'] * 0.33),
                                                         np.where((staff_util_data_latestYr['Util_Flag'] == "SUR_Q5"),
                                                                  (staff_util_data_latestYr['Non-Productive Non-Billable Hours'] * 0.66),
                                                                      staff_util_data_latestYr['Non-Productive Non-Billable Hours']
                                                ))

staff_util_data_latestYr = staff_util_data_latestYr.groupby(['Masked Code'], as_index=False).agg({'Billable Hours':'sum'
                                                                                                      ,'Productive Non-Billable Hours':'sum'
                                                                                                      ,'Non-Productive Non-Billable Hours':'sum'
                                                                                                     ,'Standard Hours':'sum'})

# derive metrics
staff_util_data_latestYr['BILL_UTIL'] = staff_util_data_latestYr['Billable Hours']/staff_util_data_latestYr['Standard Hours']
staff_util_data_latestYr['PROD_NONBILL_UTIL'] = staff_util_data_latestYr['Productive Non-Billable Hours']/staff_util_data_latestYr['Standard Hours']
staff_util_data_latestYr['NONPROD_NONBILL_UTIL'] = staff_util_data_latestYr['Non-Productive Non-Billable Hours']/staff_util_data_latestYr['Standard Hours']

staff_util_data_latestYr = staff_util_data_latestYr[['Masked Code','BILL_UTIL','PROD_NONBILL_UTIL','NONPROD_NONBILL_UTIL']]
staff_util_data_latestYr.columns = ['Masked Code', 'LatestYr_BILL_UTIL', 'LatestYr_PROD_NONBILL_UTIL', 'LatestYr_NONPROD_NONBILL_UTIL']

DATA = pd.merge(DATA, staff_util_data_latestYr, on='Masked Code', how='left')


# by lastest quarter
staff_util_data_latestQtr = staff_util_data[(staff_util_data['Util_Flag'] == "SUR_Q1") |
                                            (staff_util_data['Util_Flag'] == "SUR_Q2")]

# formulae to adjust for quarter definition
staff_util_data_latestQtr['Standard Hours'] = np.where((staff_util_data_latestQtr['Util_Flag'] == "SUR_Q1"),
                                                    (staff_util_data_latestQtr['Standard Hours'] * 0.33),
                                                         (staff_util_data_latestQtr['Standard Hours'] * 0.66)
                                                )

staff_util_data_latestQtr['Billable Hours'] = np.where((staff_util_data_latestQtr['Util_Flag'] ==  "SUR_Q1"),
                                                            (staff_util_data_latestQtr['Billable Hours'] * 0.33),
                                                                 (staff_util_data_latestQtr['Billable Hours'] * 0.66)
                                                )

staff_util_data_latestQtr['Productive Non-Billable Hours'] = np.where((staff_util_data_latestQtr['Util_Flag'] ==  "SUR_Q1"),
                                                                (staff_util_data_latestQtr['Productive Non-Billable Hours'] * 0.33),
                                                                     (staff_util_data_latestQtr['Productive Non-Billable Hours'] * 0.66)
                                                )

staff_util_data_latestQtr['Non-Productive Non-Billable Hours'] = np.where((staff_util_data_latestQtr['Util_Flag'] ==  "SUR_Q1"),
                                                                (staff_util_data_latestQtr['Non-Productive Non-Billable Hours'] * 0.33),
                                                                     (staff_util_data_latestQtr['Non-Productive Non-Billable Hours'] * 0.66)
                                                )

staff_util_data_latestQtr = staff_util_data_latestQtr.groupby(['Masked Code'], as_index=False).agg({'Billable Hours':'sum'
                                                                                                      ,'Productive Non-Billable Hours':'sum'
                                                                                                      ,'Non-Productive Non-Billable Hours':'sum'
                                                                                                     ,'Standard Hours':'sum'})

# derive metrics
staff_util_data_latestQtr['BILL_UTIL'] = staff_util_data_latestQtr['Billable Hours']/staff_util_data_latestQtr['Standard Hours']
staff_util_data_latestQtr['PROD_NONBILL_UTIL'] = staff_util_data_latestQtr['Productive Non-Billable Hours']/staff_util_data_latestQtr['Standard Hours']
staff_util_data_latestQtr['NONPROD_NONBILL_UTIL'] = staff_util_data_latestQtr['Non-Productive Non-Billable Hours']/staff_util_data_latestQtr['Standard Hours']

staff_util_data_latestQtr = staff_util_data_latestQtr[['Masked Code','BILL_UTIL','PROD_NONBILL_UTIL','NONPROD_NONBILL_UTIL']]
staff_util_data_latestQtr.columns = ['Masked Code', 'LatestQtr_BILL_UTIL', 'LatestQtr_PROD_NONBILL_UTIL', 'LatestQtr_NONPROD_NONBILL_UTIL']

DATA = pd.merge(DATA, staff_util_data_latestQtr, on='Masked Code', how='left')

print("Successfully executed Staff Utilization Data\r\n")


Successfully executed Staff Utilization Data



In [29]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Past Employment Data\\')

In [30]:

past_employment_data = pd.read_excel('Past Employment Data for Active and Inactive Practitioners.xlsx', sheet_name = "Sheet1", header=1)


# subset required columns
past_employment_data = past_employment_data[['Masked Code'
                                    ,'From', 'To', 'Exp. in Years and Months'
                                    ,'From.1', 'To.1', 'Exp. in Years and Months.1'
                                    ,'From.2', 'To.2', 'Exp. in Years and Months.2'
                                    ,'From.3', 'To.3', 'Exp. in Years and Months.3'
                                    ,'From.4', 'To.4', 'Exp. in Years and Months.4'
                                    ,'From.5', 'To.5', 'Exp. in Years and Months.5'
                                    ,'From.6', 'To.6', 'Exp. in Years and Months.6'
                                    ,'From.7', 'To.7', 'Exp. in Years and Months.7'
                                    ,'From.8', 'To.8', 'Exp. in Years and Months.8'
                                    ,'From.9', 'To.9', 'Exp. in Years and Months.9'
                                    ,'From.10', 'To.10', 'Exp. in Years and Months.10'
                                    ,'From.11', 'To.11', 'Exp. in Years and Months.11'
                                    ,'From.12', 'To.12', 'Exp. in Years and Months.12'
                                    ,'From.13', 'To.13', 'Exp. in Years and Months.13'
                                    ,'Deloitte Exp', 'Total Exp.']]

# calculate number of prior companies worked
past_employment_data = past_employment_data.assign(NUM_PRIOR_COMPANIES = past_employment_data.loc[:, 'From':'Exp. in Years and Months.13'].notna().sum(axis=1))
past_employment_data['NUM_PRIOR_COMPANIES'] = past_employment_data['NUM_PRIOR_COMPANIES']/3

# change the date format
past_employment_data[['From','To'
            ,'From.1', 'To.1'
            ,'From.2', 'To.2'
            ,'From.3', 'To.3'
            ,'From.4', 'To.4'
            ,'From.5', 'To.5'
            ,'From.6', 'To.6'
            ,'From.7', 'To.7'
            ,'From.8', 'To.8'
            ,'From.9', 'To.9'
            ,'From.10', 'To.10'
            ,'From.11', 'To.11'
            ,'From.12', 'To.12'
            ,'From.13', 'To.13'
            ]] = past_employment_data[['From','To'
                                    ,'From.1', 'To.1'
                                    ,'From.2', 'To.2'
                                    ,'From.3', 'To.3'
                                    ,'From.4', 'To.4'
                                    ,'From.5', 'To.5'
                                    ,'From.6', 'To.6'
                                    ,'From.7', 'To.7'
                                    ,'From.8', 'To.8'
                                    ,'From.9', 'To.9'
                                    ,'From.10', 'To.10'
                                    ,'From.11', 'To.11'
                                    ,'From.12', 'To.12'
                                    ,'From.13', 'To.13']].apply(pd.to_datetime)

# replace the missing dates with a default/dummy value for easier feature calculation
past_employment_data[['From','To'
            ,'From.1', 'To.1'
            ,'From.2', 'To.2'
            ,'From.3', 'To.3'
            ,'From.4', 'To.4'
            ,'From.5', 'To.5'
            ,'From.6', 'To.6'
            ,'From.7', 'To.7'
            ,'From.8', 'To.8'
            ,'From.9', 'To.9'
            ,'From.10', 'To.10'
            ,'From.11', 'To.11'
            ,'From.12', 'To.12'
            ,'From.13', 'To.13'
            ]] = past_employment_data[['From','To'
                                    ,'From.1', 'To.1'
                                    ,'From.2', 'To.2'
                                    ,'From.3', 'To.3'
                                    ,'From.4', 'To.4'
                                    ,'From.5', 'To.5'
                                    ,'From.6', 'To.6'
                                    ,'From.7', 'To.7'
                                    ,'From.8', 'To.8'
                                    ,'From.9', 'To.9'
                                    ,'From.10', 'To.10'
                                    ,'From.11', 'To.11'
                                    ,'From.12', 'To.12'
                                    ,'From.13', 'To.13']].replace(np.nan, pd.datetime(1900,1,1))

# calculate average tenure in prior companies
past_employment_data['PRIOR_EXP'] = ((past_employment_data['To'] - past_employment_data['From']) 
                          + (past_employment_data['To.1'] - past_employment_data['From.1']) 
                          + (past_employment_data['To.2'] - past_employment_data['From.2']) 
                          + (past_employment_data['To.3'] - past_employment_data['From.3']) 
                          + (past_employment_data['To.4'] - past_employment_data['From.4']) 
                          + (past_employment_data['To.5'] - past_employment_data['From.5']) 
                          + (past_employment_data['To.6'] - past_employment_data['From.6']) 
                          + (past_employment_data['To.7'] - past_employment_data['From.7']) 
                          + (past_employment_data['To.8'] - past_employment_data['From.8'])
                          + (past_employment_data['To.9'] - past_employment_data['From.9'])
                          + (past_employment_data['To.10'] - past_employment_data['From.10'])
                          + (past_employment_data['To.11'] - past_employment_data['From.11'])
                          + (past_employment_data['To.12'] - past_employment_data['From.12'])
                          + (past_employment_data['To.13'] - past_employment_data['From.13']))

past_employment_data['PRIOR_EXP'] = past_employment_data['PRIOR_EXP']/np.timedelta64(1,'Y')

past_employment_data['PRIOR_AVG_TENURE'] = past_employment_data['PRIOR_EXP']/past_employment_data['NUM_PRIOR_COMPANIES']

# create a column for Experienced/Fresher Flag
past_employment_data['IS_EXPERIENCED'] = 1

# keep only required columns
past_employment_data = past_employment_data[['Masked Code'
                                    ,'IS_EXPERIENCED'
                                    ,'NUM_PRIOR_COMPANIES'
                                    ,'PRIOR_EXP'
                                    ,'PRIOR_AVG_TENURE'
                                    ,'From'
                                    ,'From.1'
                                    ,'From.2'
                                    ,'From.3'
                                    ,'From.4'
                                    ,'From.5'
                                    ,'From.6'
                                    ,'From.7'
                                    ,'From.8'
                                    ,'From.9'
                                    ,'From.10'
                                    ,'From.11'
                                    ,'From.12'
                                    ,'From.13'
                                    ]]

# enrich with past employment data
DATA = pd.merge(DATA, past_employment_data, on='Masked Code', how='left')
DATA['IS_EXPERIENCED'] = DATA['IS_EXPERIENCED'].fillna(0)

# calculate total working experience (including Deloitte Experience)
DATA['TOTAL_WORK_EXP'] = DATA['PRIOR_EXP'] + DATA['YEARS_AT_COMPANY']

# calculate the number of companies an employee worked in last 5 years
DATA['LAST5YRS_CUTOFF_DATE'] = q1_end - pd.DateOffset(years = 5)

# replace the missing dates with a default/dummy value for easier feature calculation for freshers
DATA[['From'
,'From.1'
,'From.2'
,'From.3'
,'From.4'
,'From.5'
,'From.6'
,'From.7'
,'From.8'
,'From.9'
,'From.10'
,'From.11'
,'From.12'
,'From.13'
]] = DATA[['From'
,'From.1'
,'From.2'
,'From.3'
,'From.4'
,'From.5'
,'From.6'
,'From.7'
,'From.8'
,'From.9'
,'From.10'
,'From.11'
,'From.12'
,'From.13']].replace(np.nan, pd.datetime(1900,1,1))

DATA['LAST5YRS_CUTOFF_DATE'] = pd.to_datetime(DATA['LAST5YRS_CUTOFF_DATE']) #, errors='coerce'

DATA['LAST5YRS_COMP_WORKED'] = DATA[['From'
                            ,'From.1'
                            ,'From.2'
                            ,'From.3'
                            ,'From.4'
                            ,'From.5'
                            ,'From.6'
                            ,'From.7'
                            ,'From.8'
                            ,'From.9'
                            ,'From.10'
                            ,'From.11'
                            ,'From.12'
                            ,'From.13']].ge(DATA.loc[:, 'LAST5YRS_CUTOFF_DATE'], axis = 0).sum(axis = 1)

DATA['LAST5YRS_COMP_WORKED'] = np.where((DATA['IS_EXPERIENCED'] == 1), 
                                              (DATA['LAST5YRS_COMP_WORKED'] + 1),
                                                  DATA['LAST5YRS_COMP_WORKED'])

print("Successfully executed Past Employment Data\r\n")


Successfully executed Past Employment Data



In [31]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\LnD Data\\')

In [32]:

lnd_data = pd.read_excel('LnD Data.xlsx', sheet_name = "Sheet1")



lnd_data['Completion Date'] = pd.to_datetime(lnd_data['Completion Date'])
lnd_data = lnd_data[['Masked Code','Category','Learning Hours','Completion Date']]

# remove Compliance trainings
lnd_data = lnd_data[lnd_data['Category'] != "Compliance"]

# move the Future Skills and Industry to Technical
lnd_data['Category'][lnd_data['Category'] == "Future Skills"] = "Technical"
lnd_data['Category'][lnd_data['Category'] == "Industry"] = "Technical"

# extract year and quarter from the learning completion date
lnd_data['Quarter'] = pd.PeriodIndex(lnd_data['Completion Date'], freq='Q').strftime('Q%q')
lnd_data['Year'] = pd.PeriodIndex(lnd_data['Completion Date'], freq='Y').strftime('%Y')
lnd_data['YearQuarter'] = lnd_data['Year'] + lnd_data['Quarter']
lnd_data = lnd_data[['Masked Code', 'YearQuarter', 'Category','Learning Hours']]

# calculate learning hours
lnd_data_learning_hours = lnd_data.groupby(['Masked Code', 'YearQuarter', 'Category'], as_index=False).agg({'Learning Hours':'sum'})

# create features by category for learning_hours
lnd_data_learning_hours['Technical_Learning_hours'] = np.where((lnd_data_learning_hours['Category'] == 'Technical'), lnd_data_learning_hours['Learning Hours'], 0)
lnd_data_learning_hours['ProfLeadDev_Learning_hours'] = np.where((lnd_data_learning_hours['Category'] == 'Professional & Leadership Development'), lnd_data_learning_hours['Learning Hours'], 0)
lnd_data_learning_hours = lnd_data_learning_hours[['Masked Code', 'YearQuarter', 'Technical_Learning_hours', 'ProfLeadDev_Learning_hours']]
lnd_data_learning_hours = lnd_data_learning_hours.groupby(['Masked Code', 'YearQuarter'], as_index=False).agg({'Technical_Learning_hours':'sum'
                                                                                                          ,'ProfLeadDev_Learning_hours':'sum'})

# calculate number of courses taken
lnd_data_num_courses = lnd_data.groupby(['Masked Code', 'YearQuarter', 'Category'], as_index=False).count()
lnd_data_num_courses.columns = ['Masked Code','YearQuarter','Category','NUM_COURSES']

# create features by category for num_courses
lnd_data_num_courses['Technical_num_courses'] = np.where((lnd_data_num_courses['Category'] == 'Technical'), lnd_data_num_courses['NUM_COURSES'], 0)
lnd_data_num_courses['ProfLeadDev_num_courses'] = np.where((lnd_data_num_courses['Category'] == 'Professional & Leadership Development'), lnd_data_num_courses['NUM_COURSES'], 0)
lnd_data_num_courses = lnd_data_num_courses[['Masked Code', 'YearQuarter', 'Technical_num_courses', 'ProfLeadDev_num_courses']]
lnd_data_num_courses = lnd_data_num_courses.groupby(['Masked Code', 'YearQuarter'], as_index=False).agg({'Technical_num_courses':'sum' 
                                                                                                     ,'ProfLeadDev_num_courses':'sum'})



# enrich with lnd data
# learning_hours
lnd_data_learning_hours_lastestQtr = lnd_data_learning_hours[lnd_data_learning_hours['YearQuarter'] == yr_qtr1]
lnd_data_learning_hours_lastestQtr = lnd_data_learning_hours_lastestQtr[['Masked Code', 'Technical_Learning_hours', 'ProfLeadDev_Learning_hours']]
lnd_data_learning_hours_lastestQtr.columns = ['Masked Code', 'LatestQtrTechLearnHours', 'LatestQtrProfLeadDevLearnHours']
DATA = pd.merge(DATA, lnd_data_learning_hours_lastestQtr, on='Masked Code', how='left')

lnd_data_learning_hours_latestYr = lnd_data_learning_hours[(lnd_data_learning_hours['YearQuarter'] == yr_qtr1) | 
                                                          (lnd_data_learning_hours['YearQuarter'] == yr_qtr2) |
                                                          (lnd_data_learning_hours['YearQuarter'] == yr_qtr3) |
                                                          (lnd_data_learning_hours['YearQuarter'] == yr_qtr4)]
lnd_data_learning_hours_latestYr = lnd_data_learning_hours_latestYr[['Masked Code', 'Technical_Learning_hours', 'ProfLeadDev_Learning_hours']]
lnd_data_learning_hours_latestYr.columns = ['Masked Code', 'LatestYrTechLearnHours', 'LatestYrProfLeadDevLearnHours']
lnd_data_learning_hours_latestYr = lnd_data_learning_hours_latestYr.groupby(['Masked Code'], as_index=False).agg({'LatestYrTechLearnHours':'sum'
                                                                                                                      ,'LatestYrProfLeadDevLearnHours':'sum'})

DATA = pd.merge(DATA, lnd_data_learning_hours_latestYr, on='Masked Code', how='left')

DATA['LatestQtrTechLearnHours'] = DATA['LatestQtrTechLearnHours'].fillna(0)
DATA['LatestQtrProfLeadDevLearnHours'] = DATA['LatestQtrProfLeadDevLearnHours'].fillna(0)
DATA['LatestYrTechLearnHours'] = DATA['LatestYrTechLearnHours'].fillna(0)
DATA['LatestYrProfLeadDevLearnHours'] = DATA['LatestYrProfLeadDevLearnHours'].fillna(0)

# number of courses
lnd_data_num_courses_lastestQtr = lnd_data_num_courses[lnd_data_num_courses['YearQuarter'] == yr_qtr1]
lnd_data_num_courses_lastestQtr = lnd_data_num_courses_lastestQtr[['Masked Code', 'Technical_num_courses', 'ProfLeadDev_num_courses']]
lnd_data_num_courses_lastestQtr.columns = ['Masked Code', 'LatestQtrTechNumCourses', 'LatestQtrProfLeadDevNumCourses']
DATA = pd.merge(DATA, lnd_data_num_courses_lastestQtr, on='Masked Code', how='left')

lnd_data_num_courses_latestYr = lnd_data_num_courses[(lnd_data_num_courses['YearQuarter'] == yr_qtr1) | 
                                                          (lnd_data_num_courses['YearQuarter'] == yr_qtr2) |
                                                          (lnd_data_num_courses['YearQuarter'] == yr_qtr3) |
                                                          (lnd_data_num_courses['YearQuarter'] == yr_qtr4)]
lnd_data_num_courses_latestYr = lnd_data_num_courses_latestYr[['Masked Code', 'Technical_num_courses', 'ProfLeadDev_num_courses']]
lnd_data_num_courses_latestYr.columns = ['Masked Code', 'LatestYrTechNumCourses', 'LatestYrProfLeadDevNumCourses']
lnd_data_num_courses_latestYr = lnd_data_num_courses_latestYr.groupby(['Masked Code'], as_index=False).agg({'LatestYrTechNumCourses':'sum'
                                                                                                                      ,'LatestYrProfLeadDevNumCourses':'sum'})

DATA = pd.merge(DATA, lnd_data_num_courses_latestYr, on='Masked Code', how='left')

DATA['LatestQtrTechNumCourses'] = DATA['LatestQtrTechNumCourses'].fillna(0)
DATA['LatestQtrProfLeadDevNumCourses'] = DATA['LatestQtrProfLeadDevNumCourses'].fillna(0)
DATA['LatestYrTechNumCourses'] = DATA['LatestYrTechNumCourses'].fillna(0)
DATA['LatestYrProfLeadDevNumCourses'] = DATA['LatestYrProfLeadDevNumCourses'].fillna(0)

print("Successfully executed LnD Data\r\n")


Successfully executed LnD Data



In [33]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\RnR Data\\')

In [34]:

rnr_data = pd.read_excel('RnR Data.xlsx', sheet_name = "Sheet1")


rnr_data['CompletionDate'] = pd.to_datetime(rnr_data['Nomination Date'])
rnr_data = rnr_data[['Masked Code', 'Type', 'Reward name', 'Nomination Date', 'Amount(Rs)']]

# keep the records of only employees having rewards and recognitions
rnr_data = rnr_data[(rnr_data['Type'] == 'Reward') | (rnr_data['Type'] == 'Recognition')]

# extract year and quarter from the Nomination Date
rnr_data['Quarter'] = pd.PeriodIndex(rnr_data['Nomination Date'], freq='Q').strftime('Q%q')
rnr_data['Year'] = pd.PeriodIndex(rnr_data['Nomination Date'], freq='Y').strftime('%Y')
rnr_data['YearQuarter'] = rnr_data['Year'] + rnr_data['Quarter']
rnr_data = rnr_data[['Masked Code', 'YearQuarter', 'Type','Reward name', 'Amount(Rs)']]

# calculate amount received from rewards/recognitions (look for bins later)
rnr_amount_data = rnr_data.groupby(['Masked Code', 'YearQuarter'], as_index=False).agg({'Amount(Rs)':'sum'})

# calculate count of rewards/recognitions
rnr_data = rnr_data[['Masked Code', 'YearQuarter', 'Type', 'Reward name']]
rnr_data = rnr_data.groupby(['Masked Code', 'YearQuarter', 'Type'], as_index=False).count()
rnr_data.columns = ['Masked Code', 'YearQuarter', 'Type', 'Count']

rnr_data['NumOfRewards'] = np.where((rnr_data['Type'] == 'Reward'), rnr_data['Count'], 0)
rnr_data['NumOfRecognitions'] = np.where((rnr_data['Type'] == 'Recognition'), rnr_data['Count'], 0)
rnr_data = rnr_data.groupby(['Masked Code', 'YearQuarter'], as_index=False).agg({'NumOfRewards':'sum', 'NumOfRecognitions':'sum'})

# calculate total RnR
rnr_data['TotalRnR'] = rnr_data['NumOfRewards'] + rnr_data['NumOfRecognitions']

# derive the columns Is Recognized and Is Rewarded
rnr_data['IsRewarded'] = np.where((rnr_data['NumOfRewards'] >= 1), 1, 0)
rnr_data['IsRecognized'] = np.where((rnr_data['NumOfRecognitions'] >= 1), 1, 0)

# enrich with RnR data
# RnR awards
rnr_data = rnr_data[rnr_data['YearQuarter'] == yr_qtr1]
rnr_data = rnr_data[['Masked Code', 'NumOfRewards', 'NumOfRecognitions', 'TotalRnR', 'IsRewarded', 'IsRecognized']]
DATA = pd.merge(DATA, rnr_data, on='Masked Code', how='left')

# RnR amount
rnr_amount_data = rnr_amount_data[rnr_amount_data['YearQuarter'] == yr_qtr1]
rnr_amount_data = rnr_amount_data[['Masked Code', 'Amount(Rs)']]
DATA = pd.merge(DATA, rnr_amount_data, on='Masked Code', how='left')

# fill missing values with 0
DATA['NumOfRewards'] = DATA['NumOfRewards'].fillna(0)
DATA['NumOfRecognitions'] = DATA['NumOfRecognitions'].fillna(0)
DATA['TotalRnR'] = DATA['TotalRnR'].fillna(0)
DATA['IsRewarded'] = DATA['IsRewarded'].fillna(0)
DATA['IsRecognized'] = DATA['IsRecognized'].fillna(0)
DATA['Amount(Rs)'] = DATA['Amount(Rs)'].fillna(0)

print("Successfully executed RnR Data\r\n")


Successfully executed RnR Data



In [35]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Snapshot Data\\')

In [36]:

snapshot_results_data = pd.read_excel('Performance Snapshot Data.xlsx' , sheet_name ='Sheet1')


# subset required columns
snapshot_results_data = snapshot_results_data[['Masked Emp. Code.', 'Created Date','Q1 Answer Value','Q2 Answer Value','Q3 Answer Value', 'Q4 Answer Value']]
snapshot_results_data.columns = ['Masked Code','Date Created' ,'Q1_ans','Q2_ans','Q3_ans','Q4_ans']

snapshot_results_data['Date Created'] = pd.to_datetime(snapshot_results_data['Date Created'])

# extract year and quarter from the Snapshot Creation Date
snapshot_results_data['Quarter'] = pd.PeriodIndex(snapshot_results_data['Date Created'], freq='Q').strftime('Q%q')
snapshot_results_data['Year'] = pd.PeriodIndex(snapshot_results_data['Date Created'], freq='Y').strftime('%Y')
snapshot_results_data['YearQuarter'] = snapshot_results_data['Year'] + snapshot_results_data['Quarter']
snapshot_results_data = snapshot_results_data[['Masked Code', 'YearQuarter','Q1_ans','Q2_ans','Q3_ans','Q4_ans']]

snapshot_results_data['Q1_ans_weightage'] = np.where((snapshot_results_data['Q1_ans'] == 5), 100,
                                                 np.where((snapshot_results_data['Q1_ans'] == 4), 80,
                                                          np.where((snapshot_results_data['Q1_ans'] == 3), 60,
                                                                   np.where((snapshot_results_data['Q1_ans'] == 2), 40,
                                                                            np.where((snapshot_results_data['Q1_ans'] == 1), 20,
                                                                            0)))))

snapshot_results_data['Q2_ans_weightage'] = np.where((snapshot_results_data['Q2_ans'] == 5), 100,
                                                 np.where((snapshot_results_data['Q2_ans'] == 4), 80,
                                                          np.where((snapshot_results_data['Q2_ans'] == 3), 60,
                                                                   np.where((snapshot_results_data['Q2_ans'] == 2), 40,
                                                                            np.where((snapshot_results_data['Q2_ans'] == 1), 20,
                                                                            0)))))

snapshot_results_data['Q3_ans_weightage'] = np.where((snapshot_results_data['Q3_ans'] == 2), 100,0)

snapshot_results_data['Q4_ans_weightage'] = np.where((snapshot_results_data['Q4_ans'] == 1), 100,0)

snapshot_results_data['snapshot_composite_score'] = 0.25* (snapshot_results_data['Q1_ans_weightage'] + snapshot_results_data['Q2_ans_weightage']+ snapshot_results_data['Q3_ans_weightage']+snapshot_results_data['Q4_ans_weightage'])

snapshot_results_data = snapshot_results_data[['Masked Code', 'YearQuarter', 'snapshot_composite_score']]


# import Firm Contribution Snapshot Results data
snapshot_fi_results_data = pd.read_excel('Firm Contribution Snapshot Data.xlsx' , sheet_name ='Sheet1')



# subset required columns
snapshot_fi_results_data = snapshot_fi_results_data[['Masked emp. Code.', 'Created Date','Q1 Answer Value','Q2 Answer Value']]
snapshot_fi_results_data.columns = ['Masked Code','Date Created' ,'Q1_ans','Q2_ans']

snapshot_fi_results_data['Date Created'] = pd.to_datetime(snapshot_fi_results_data['Date Created'])

# extract year and quarter from the Snapshot Creation Date
snapshot_fi_results_data['Quarter'] = pd.PeriodIndex(snapshot_fi_results_data['Date Created'], freq='Q').strftime('Q%q')
snapshot_fi_results_data['Year'] = pd.PeriodIndex(snapshot_fi_results_data['Date Created'], freq='Y').strftime('%Y')
snapshot_fi_results_data['YearQuarter'] = snapshot_fi_results_data['Year'] + snapshot_fi_results_data['Quarter']
snapshot_fi_results_data = snapshot_fi_results_data[['Masked Code', 'YearQuarter','Q1_ans','Q2_ans']]

snapshot_fi_results_data['Q1_ans_weightage'] = np.where((snapshot_fi_results_data['Q1_ans'] == 5), 100,
                                                 np.where((snapshot_fi_results_data['Q1_ans'] == 4), 80,
                                                          np.where((snapshot_fi_results_data['Q1_ans'] == 3), 60,
                                                                   np.where((snapshot_fi_results_data['Q1_ans'] == 2), 40,
                                                                            np.where((snapshot_fi_results_data['Q1_ans'] == 1), 20,
                                                                            0)))))

snapshot_fi_results_data['Q2_ans_weightage'] = np.where((snapshot_fi_results_data['Q2_ans'] == 5), 100,
                                                 np.where((snapshot_fi_results_data['Q2_ans'] == 4), 80,
                                                          np.where((snapshot_fi_results_data['Q2_ans'] == 3), 60,
                                                                   np.where((snapshot_fi_results_data['Q2_ans'] == 2), 40,
                                                                            np.where((snapshot_fi_results_data['Q2_ans'] == 1), 20,
                                                                            0)))))

snapshot_fi_results_data['snapshot_composite_score'] = 0.5* (snapshot_fi_results_data['Q1_ans_weightage'] + snapshot_fi_results_data['Q2_ans_weightage'])

snapshot_fi_results_data = snapshot_fi_results_data[['Masked Code', 'YearQuarter', 'snapshot_composite_score']]

# append performance and firm contribution snapshot results
snapshot_results_data = pd.concat([snapshot_results_data, snapshot_fi_results_data], axis=0)

# normalising data
snapshot_results_data['snapshot_composite_score'] = (snapshot_results_data['snapshot_composite_score'])/100

# enrich with the snapshot results data
# by latest quarter
snapshot_results_data_latestQtr = snapshot_results_data[snapshot_results_data['YearQuarter'] == yr_qtr1]
snapshot_results_data_latestQtr = snapshot_results_data_latestQtr[['Masked Code', 'snapshot_composite_score']]
snapshot_results_data_latestQtr.columns = ['Masked Code','LatestQtr_snapshot_composite_score']
snapshot_results_data_latestQtr = snapshot_results_data_latestQtr.groupby(['Masked Code'], as_index=False).agg({'LatestQtr_snapshot_composite_score':'mean'})
DATA = pd.merge(DATA, snapshot_results_data_latestQtr, on='Masked Code', how='left')

# by latest year
# filter the latest year
snapshot_results_data_latestYr = snapshot_results_data[(snapshot_results_data['YearQuarter'] == yr_qtr1) | 
                                                      (snapshot_results_data['YearQuarter'] == yr_qtr2) |
                                                      (snapshot_results_data['YearQuarter'] == yr_qtr3) |
                                                      (snapshot_results_data['YearQuarter'] == yr_qtr4)]

snapshot_results_data_latestYr = snapshot_results_data_latestYr[['Masked Code', 'snapshot_composite_score']]
snapshot_results_data_latestYr.columns = ['Masked Code','LatestYr_snapshot_composite_score']
snapshot_results_data_latestYr = snapshot_results_data_latestYr.groupby(['Masked Code'], as_index=False).agg({'LatestYr_snapshot_composite_score':'mean'})
DATA= pd.merge(DATA, snapshot_results_data_latestYr, on='Masked Code', how='left')

print("Successfully executed Snapshot Results Data\r\n")


Successfully executed Snapshot Results Data



In [37]:
snapshot_results_data['YearQuarter'].unique()

array(['2020Q3', '2020Q2', '2020Q4', '2020Q1', '2019Q2', '2019Q4',
       '2019Q3', '2019Q1', '2018Q2', '2018Q4', '2018Q3', '2018Q1',
       '2017Q1', '2017Q2', '2017Q4', '2017Q3', '2016Q4', '2016Q3'],
      dtype=object)

In [38]:

# import snapshot data
snapshot_perf_data = pd.read_excel('Performance Snapshot Data.xlsx' , sheet_name ='Sheet1')
snapshot_perf_data = snapshot_perf_data[['Masked Emp. Code.', 'Created Date', 'Project Start Date', 'Project End Date', 'Status', 'Snapshot Type', 'Project Hours  ']]
snapshot_perf_data.columns = ['Masked Code','Date Created','Project Start Date', 'Project End Date', 'Status', 'Snapshot Name','Project Hours']

snapshot_firm_data = pd.read_excel('Firm Contribution Snapshot Data.xlsx' , sheet_name ='Sheet1')
snapshot_firm_data = snapshot_firm_data[['Masked emp. Code.', 'Created Date', 'Project Start Date', 'Project End Date', 'Status', 'Snapshot Type', 'Project Hours  ']]
snapshot_firm_data.columns = ['Masked Code','Date Created','Project Start Date', 'Project End Date', 'Status', 'Snapshot Name','Project Hours']

# append performance and firm contribution snapshots
snapshot_data = pd.concat([snapshot_perf_data, snapshot_firm_data], axis=0)

snapshot_project_data = snapshot_data[['Masked Code', 'Project Start Date', 'Project End Date']]
snapshot_data = snapshot_data[['Masked Code', 'Date Created', 'Status', 'Snapshot Name', 'Project Hours']]

# renaming the columns
snapshot_project_data.columns = ['Masked Code', 'Project Start Date', 'Project End Date']

# format the dates
snapshot_project_data['Project Start Date'] = pd.to_datetime(snapshot_project_data['Project Start Date'])
snapshot_project_data['Project End Date'] = pd.to_datetime(snapshot_project_data['Project End Date'])

if min(((snapshot_project_data['Project End Date'] - snapshot_project_data['Project Start Date'])/
                                                                  np.timedelta64(1, 'D')).astype(int)):
    print("Data Quality Issue - Snapshot Data has negative project days\r\n")


    # logic to calculate the project days by quarter and by latest year
snapshot_project_data['QEQSDiff_QB4'] = (q4_end - q4_start) + timedelta(days=1)
snapshot_project_data['PEQSDiff_QB4'] = (snapshot_project_data['Project End Date']  - q4_start) + timedelta(days=1)
snapshot_project_data['QEPSDiff_QB4'] = (q4_end - snapshot_project_data['Project Start Date']) + timedelta(days=1)
snapshot_project_data['PEPSDiff_QB4'] = (snapshot_project_data['Project End Date']  - snapshot_project_data['Project Start Date']) + timedelta(days=1)

snapshot_project_data['QEQSDiff_QB3'] = (q3_end - q3_start) + timedelta(days=1)
snapshot_project_data['PEQSDiff_QB3'] = (snapshot_project_data['Project End Date']  - q3_start) + timedelta(days=1)
snapshot_project_data['QEPSDiff_QB3'] = (q3_end - snapshot_project_data['Project Start Date']) + timedelta(days=1)
snapshot_project_data['PEPSDiff_QB3'] = (snapshot_project_data['Project End Date']  - snapshot_project_data['Project Start Date']) + timedelta(days=1)

snapshot_project_data['QEQSDiff_QB2'] = (q2_end - q2_start) + timedelta(days=1)
snapshot_project_data['PEQSDiff_QB2'] = (snapshot_project_data['Project End Date']  - q2_start) + timedelta(days=1)
snapshot_project_data['QEPSDiff_QB2'] = (q2_end - snapshot_project_data['Project Start Date']) + timedelta(days=1)
snapshot_project_data['PEPSDiff_QB2'] = (snapshot_project_data['Project End Date']  - snapshot_project_data['Project Start Date']) + timedelta(days=1)

snapshot_project_data['QEQSDiff_QB1'] = (q1_end - q1_start) + timedelta(days=1)
snapshot_project_data['PEQSDiff_QB1'] = (snapshot_project_data['Project End Date']  - q1_start) + timedelta(days=1)
snapshot_project_data['QEPSDiff_QB1'] = (q1_end - snapshot_project_data['Project Start Date']) + timedelta(days=1)
snapshot_project_data['PEPSDiff_QB1'] = (snapshot_project_data['Project End Date']  - snapshot_project_data['Project Start Date']) + timedelta(days=1)

# calculate the number of days an employee has worked in each quarter
# work on 4th quarter
cols_project_QB4 = ['Project Start Date', 'Project End Date', 'QEQSDiff_QB4',  'PEQSDiff_QB4', 'QEPSDiff_QB4', 'PEPSDiff_QB4']
snapshot_project_data[yr_qtr4] = quarter_days_allocator(snapshot_project_data, cols_project_QB4, q4_start, q4_end)

# work on 3rd quarter
cols_project_QB3 = ['Project Start Date', 'Project End Date', 'QEQSDiff_QB3',  'PEQSDiff_QB3', 'QEPSDiff_QB3', 'PEPSDiff_QB3']
snapshot_project_data[yr_qtr3] = quarter_days_allocator(snapshot_project_data, cols_project_QB3, q3_start, q3_end)

# work on 2nd quarter
cols_project_QB2 = ['Project Start Date', 'Project End Date', 'QEQSDiff_QB2',  'PEQSDiff_QB2', 'QEPSDiff_QB2', 'PEPSDiff_QB2']
snapshot_project_data[yr_qtr2] = quarter_days_allocator(snapshot_project_data, cols_project_QB2, q2_start, q2_end)

# work on 1st quarter
cols_project_QB1 = ['Project Start Date', 'Project End Date', 'QEQSDiff_QB1',  'PEQSDiff_QB1', 'QEPSDiff_QB1', 'PEPSDiff_QB1']
snapshot_project_data[yr_qtr1] = quarter_days_allocator(snapshot_project_data, cols_project_QB1, q1_start, q1_end)



# subset only required columns
snapshot_project_data = snapshot_project_data[['Masked Code',yr_qtr4, yr_qtr3, yr_qtr2, yr_qtr1]]

# converting from days to int
snapshot_project_data[yr_qtr4] = (snapshot_project_data[yr_qtr4] / np.timedelta64(1, 'D')).astype(int)
snapshot_project_data[yr_qtr3] = (snapshot_project_data[yr_qtr3] / np.timedelta64(1, 'D')).astype(int)
snapshot_project_data[yr_qtr2] = (snapshot_project_data[yr_qtr2] / np.timedelta64(1, 'D')).astype(int)
snapshot_project_data[yr_qtr1] = (snapshot_project_data[yr_qtr1] / np.timedelta64(1, 'D')).astype(int)

# grouping by employees
snapshot_project_data = snapshot_project_data.groupby(['Masked Code'], as_index=False).agg({yr_qtr4:'sum'
                                                                                      ,yr_qtr3:'sum'                                                                           
                                                                                      ,yr_qtr2:'sum'
                                                                                      ,yr_qtr1:'sum'})

# unpivot the dataset
snapshot_project_data = snapshot_project_data.melt(id_vars=['Masked Code'], var_name='YearQuarter', value_name='ProjectDays')

# logic to calculate the count of snapshots
# format the dates
snapshot_data['Date Created'] = pd.to_datetime(snapshot_data['Date Created'])

# calculate the features on number of snapshots
# extract year and quarter from the Snapshot Creation Date
snapshot_data['Quarter'] = pd.PeriodIndex(snapshot_data['Date Created'], freq='Q').strftime('Q%q')
snapshot_data['Year'] = pd.PeriodIndex(snapshot_data['Date Created'], freq='Y').strftime('%Y')
snapshot_data['YearQuarter'] = snapshot_data['Year'] + snapshot_data['Quarter']
snapshot_data = snapshot_data[['Masked Code', 'YearQuarter','Status', 'Snapshot Name','Project Hours']]

# calculate number of performance snapshots and other snapshots
snapshot_data['PerfSnapshot'] = np.where((snapshot_data['Snapshot Name'] == 'Performance Snapshot'), 1, 0)
snapshot_data['OtherSnapshot'] = np.where((snapshot_data['Snapshot Name'] == 'Firm Contribution Snapshot'), 1, 0)
snapshot_data = snapshot_data.groupby(['Masked Code', 'YearQuarter', 'Status'], as_index=False).agg({'PerfSnapshot':'sum', 'OtherSnapshot':'sum'})

# cap the snapshot counts
snapshot_data['PerfSnapshot'] = np.where((snapshot_data['PerfSnapshot'] > 15), 15, snapshot_data['PerfSnapshot'])

# calculate total number of snapshots
snapshot_data['TotalSnapshot'] = snapshot_data['PerfSnapshot'] + snapshot_data['OtherSnapshot']


# enrich with the snapshot data
# filter and merge the snapshot project days data
# by latest quarter
snapshot_project_data_latestQtr = snapshot_project_data[snapshot_project_data['YearQuarter'] == yr_qtr1]
snapshot_project_data_latestQtr = snapshot_project_data_latestQtr[['Masked Code', 'ProjectDays']]
snapshot_project_data_latestQtr.columns = ['Masked Code','LatestQtrProjectDays']
DATA = pd.merge(DATA, snapshot_project_data_latestQtr, on='Masked Code', how='left')

# by latest year
snapshot_project_data_latestYr = snapshot_project_data[(snapshot_project_data['YearQuarter'] == yr_qtr1) | 
                                                      (snapshot_project_data['YearQuarter'] == yr_qtr2) |
                                                      (snapshot_project_data['YearQuarter'] == yr_qtr3) |
                                                      (snapshot_project_data['YearQuarter'] == yr_qtr4)]
snapshot_project_data_latestYr = snapshot_project_data_latestYr[['Masked Code', 'ProjectDays']]
snapshot_project_data_latestYr.columns = ['Masked Code','LatestYrProjectDays']
snapshot_project_data_latestYr = snapshot_project_data_latestYr.groupby(['Masked Code'], as_index=False).agg({'LatestYrProjectDays':'sum'})
DATA = pd.merge(DATA, snapshot_project_data_latestYr, on='Masked Code', how='left')

# filter and merge the snapshot count data
# by latest quarter
snapshot_data_latestQtr = snapshot_data[snapshot_data['YearQuarter'] == yr_qtr1]
snapshot_data_latestQtr = snapshot_data_latestQtr[['Masked Code', 'PerfSnapshot', 'OtherSnapshot', 'TotalSnapshot']]

snapshot_data_latestQtr.columns = ['Masked Code', 'LatestQtrPerfSnapshot', 'LatestQtrOtherSnapshot', 'LatestQtrTotalSnapshot']
snapshot_data_latestQtr = snapshot_data_latestQtr.groupby(['Masked Code'], as_index=False).agg({'LatestQtrPerfSnapshot':'sum'
                                                                                                      ,'LatestQtrOtherSnapshot':'sum'
                                                                                                      ,'LatestQtrTotalSnapshot':'sum'})

DATA = pd.merge(DATA, snapshot_data_latestQtr, on='Masked Code', how='left')

# by latest year
snapshot_data_latestYr = snapshot_data[(snapshot_data['YearQuarter'] == yr_qtr1) | 
                                          (snapshot_data['YearQuarter'] == yr_qtr2) |
                                          (snapshot_data['YearQuarter'] == yr_qtr3) |
                                          (snapshot_data['YearQuarter'] == yr_qtr4)]
snapshot_data_latestYr = snapshot_data_latestYr[['Masked Code', 'PerfSnapshot', 'OtherSnapshot', 'TotalSnapshot']]
snapshot_data_latestYr.columns = ['Masked Code', 'LatestYrPerfSnapshot', 'LatestYrOtherSnapshot', 'LatestYrTotalSnapshot']
snapshot_data_latestYr = snapshot_data_latestYr.groupby(['Masked Code'], as_index=False).agg({'LatestYrPerfSnapshot':'sum'
                                                                                                    ,'LatestYrOtherSnapshot':'sum'
                                                                                                    ,'LatestYrTotalSnapshot':'sum'})

DATA = pd.merge(DATA, snapshot_data_latestYr, on='Masked Code', how='left')

# fill the missing values with 0
DATA['LatestQtrPerfSnapshot'] = DATA['LatestQtrPerfSnapshot'].fillna(0)
DATA['LatestQtrOtherSnapshot'] = DATA['LatestQtrOtherSnapshot'].fillna(0)
DATA['LatestQtrTotalSnapshot'] = DATA['LatestQtrTotalSnapshot'].fillna(0)
DATA['LatestYrPerfSnapshot'] = DATA['LatestYrPerfSnapshot'].fillna(0)
DATA['LatestYrOtherSnapshot'] = DATA['LatestYrOtherSnapshot'].fillna(0)
DATA['LatestYrTotalSnapshot'] = DATA['LatestYrTotalSnapshot'].fillna(0)

print("Successfully executed Snapshot Data\r\n")


Successfully executed Snapshot Data



In [39]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Leave Ledger Data\\')

In [40]:

leave_data_active = pd.read_excel('Leave Ledger for Active.xlsx' , sheet_name ='Sheet1')
leave_data_inactive = pd.read_excel('Leave Ledger for Inactive.xlsx' , sheet_name ='Sheet1')



# subset required columns
leave_data_active = leave_data_active[['Masked Code', 'Leave Type','From Date','To Date', 'Total No. Of Leave Days / Hours','Status']]
leave_data_active.columns = ['Masked Code', 'Leave Type','From Date','To Date','TotalLeaveDays','Status']

leave_data_inactive = leave_data_inactive[['Masked Code', 'Leave Type','From Date','To Date','Total No. Of Leave Days / Hours','Status']]
leave_data_inactive.columns = ['Masked Code', 'Leave Type','From Date','To Date','TotalLeaveDays','Status']

# append the 2 datasets
leave_data = pd.concat([leave_data_active, leave_data_inactive], axis=0)

# format the date
leave_data['From Date'] = pd.to_datetime(leave_data['From Date'])
leave_data['To Date'] = pd.to_datetime(leave_data['To Date'])

# filter leaves requests for leave utilization calculation
leave_data_request = leave_data[(leave_data['Leave Type']== 'LE-Request') | 
                            (leave_data['Leave Type']== 'SPL-Request')|
                            (leave_data['Leave Type']== 'HTO-Request')|
                            (leave_data['Leave Type']== 'LWP-Request')|
                            (leave_data['Leave Type']== 'ML-Request') |
                            (leave_data['Leave Type']== 'PL-Request') |
                            (leave_data['Leave Type']== 'BL-Request') |
                            (leave_data['Leave Type']== 'RTL-Request')]

#  separate approved and rejected leaves data 
leave_approved_data = leave_data_request[leave_data_request['Status']== 'Approved']
leave_declined_data = leave_data_request[leave_data_request['Status']== 'Rejected']

# calculate the count of rejected leaves
leave_declined_data = leave_declined_data[['Masked Code','From Date']]

# extract year and quarter from the leave request from date
leave_declined_data['Quarter'] = pd.PeriodIndex(leave_declined_data['From Date'], freq='Q').strftime('Q%q')
leave_declined_data['Year'] = pd.PeriodIndex(leave_declined_data['From Date'], freq='Y').strftime('%Y')
leave_declined_data['YearQuarter'] = leave_declined_data['Year'] + leave_declined_data['Quarter']

leave_declined_data = leave_declined_data[['Masked Code', 'YearQuarter']]

# count the declined leave requests
leave_declined_data['CountOfRejections'] = 1
leave_declined_data = leave_declined_data.groupby(['Masked Code', 'YearQuarter'], as_index=False).count()



leave_approved_data['LQEQSDiff_QB4'] = (q4_end - q4_start) + timedelta(days=1)
leave_approved_data['LEQSDiff_QB4'] = (leave_approved_data['To Date']  - q4_start) + timedelta(days=1)
leave_approved_data['QELSDiff_QB4'] = (q4_end - leave_approved_data['From Date']) + timedelta(days=1)
leave_approved_data['LELSDiff_QB4'] = (leave_approved_data['To Date']  - leave_approved_data['From Date']) + timedelta(days=1)

leave_approved_data['LQEQSDiff_QB3'] = (q3_end - q3_start) + timedelta(days=1)
leave_approved_data['LEQSDiff_QB3'] = (leave_approved_data['To Date']  - q3_start) + timedelta(days=1)
leave_approved_data['QELSDiff_QB3'] = (q3_end - leave_approved_data['From Date']) + timedelta(days=1)
leave_approved_data['LELSDiff_QB3'] = (leave_approved_data['To Date']  - leave_approved_data['From Date']) + timedelta(days=1)

leave_approved_data['LQEQSDiff_QB2'] = (q2_end - q2_start) + timedelta(days=1)
leave_approved_data['LEQSDiff_QB2'] = (leave_approved_data['To Date']  - q2_start) + timedelta(days=1)
leave_approved_data['QELSDiff_QB2'] = (q2_end - leave_approved_data['From Date']) + timedelta(days=1)
leave_approved_data['LELSDiff_QB2'] = (leave_approved_data['To Date']  - leave_approved_data['From Date']) + timedelta(days=1)

leave_approved_data['LQEQSDiff_QB1'] = (q1_end - q1_start) + timedelta(days=1)
leave_approved_data['LEQSDiff_QB1'] = (leave_approved_data['To Date']  - q1_start) + timedelta(days=1)
leave_approved_data['QELSDiff_QB1'] = (q1_end - leave_approved_data['From Date']) + timedelta(days=1)
leave_approved_data['LELSDiff_QB1'] = (leave_approved_data['To Date']  - leave_approved_data['From Date']) + timedelta(days=1)

# calculate the number of days an employee has utilized leaves in each quarter
# leave on 1st quarter
cols_leave_QB4 = ['From Date', 'To Date', 'LQEQSDiff_QB4',  'LEQSDiff_QB4', 'QELSDiff_QB4', 'LELSDiff_QB4']
leave_approved_data[yr_qtr4] = quarter_days_allocator(leave_approved_data, cols_leave_QB4, q4_start, q4_end)

# leave on 2nd quarter 2019
cols_leave_QB3 = ['From Date', 'To Date', 'LQEQSDiff_QB3',  'LEQSDiff_QB3', 'QELSDiff_QB3', 'LELSDiff_QB3']
leave_approved_data[yr_qtr3] = quarter_days_allocator(leave_approved_data, cols_leave_QB3, q3_start, q3_end)

# leave on 3rd quarter 2019
cols_leave_QB2 = ['From Date', 'To Date', 'LQEQSDiff_QB2',  'LEQSDiff_QB2', 'QELSDiff_QB2', 'LELSDiff_QB2']
leave_approved_data[yr_qtr2] = quarter_days_allocator(leave_approved_data, cols_leave_QB2, q2_start, q2_end)

# leave on 4th quarter 2019
cols_leave_QB1 = ['From Date', 'To Date', 'LQEQSDiff_QB1',  'LEQSDiff_QB1', 'QELSDiff_QB1', 'LELSDiff_QB1']
leave_approved_data[yr_qtr1] = quarter_days_allocator(leave_approved_data, cols_leave_QB1, q1_start, q1_end)



# selecting required columns
leave_approved_data = leave_approved_data[['Masked Code', 'Leave Type', 'TotalLeaveDays',yr_qtr4,yr_qtr3,yr_qtr2,yr_qtr1]]

# converting from days to int
leave_approved_data[yr_qtr4] = (leave_approved_data[yr_qtr4] / np.timedelta64(1, 'D')).astype(int)
leave_approved_data[yr_qtr3] = (leave_approved_data[yr_qtr3] / np.timedelta64(1, 'D')).astype(int)
leave_approved_data[yr_qtr2] = (leave_approved_data[yr_qtr2] / np.timedelta64(1, 'D')).astype(int)
leave_approved_data[yr_qtr1] = (leave_approved_data[yr_qtr1] / np.timedelta64(1, 'D')).astype(int)

# grouping by employees
leave_approved_data = leave_approved_data.groupby(['Masked Code','Leave Type'], as_index=False).agg({yr_qtr4:'sum'
                                                                                                  ,yr_qtr3:'sum'                                                                           
                                                                                                  ,yr_qtr2:'sum'
                                                                                                  ,yr_qtr1:'sum'})

# unpivot the dataset
leave_approved_data = leave_approved_data.melt(id_vars=['Masked Code','Leave Type'], var_name='YearQuarter', value_name='RequestUtilLeaves')

leave_approved_data[['LeaveCode','LeaveRequest']] = leave_approved_data['Leave Type'].str.split("-", expand=True) 

# create a key
leave_approved_data['Key'] = leave_approved_data['Masked Code'] + leave_approved_data['LeaveCode'] + leave_approved_data['YearQuarter']


# filter cancelled leaves requests for leave utilization calculation
leave_cancelled_data = leave_data[(leave_data['Leave Type']== 'LE-Cancellation') | 
                            (leave_data['Leave Type']== 'SPL-Cancellation')|
                            (leave_data['Leave Type']== 'HTO-Cancellation')|
                            (leave_data['Leave Type']== 'LWP-Cancellation')|
                            (leave_data['Leave Type']== 'ML-Cancellation') |
                            (leave_data['Leave Type']== 'PL-Cancellation') |
                            (leave_data['Leave Type']== 'BL-Cancellation') |
                            (leave_data['Leave Type']== 'RTL-Cancellation')]

#  separate approved and rejected leaves data 
leave_cancelled_data = leave_cancelled_data[leave_cancelled_data['Status']== 'Approved']

# calculate the date differences (Saturday and Sunday included)
leave_cancelled_data['LCQEQSDiff_QB4'] = (q4_end - q4_start) + timedelta(days=1)
leave_cancelled_data['LCEQSDiff_QB4'] = (leave_cancelled_data['To Date']  - q4_start) + timedelta(days=1)
leave_cancelled_data['QELCSDiff_QB4'] = (q4_end - leave_cancelled_data['From Date']) + timedelta(days=1)
leave_cancelled_data['LCELCSDiff_QB4'] = (leave_cancelled_data['To Date']  - leave_cancelled_data['From Date']) + timedelta(days=1)

leave_cancelled_data['LCQEQSDiff_QB3'] = (q3_end - q3_start) + timedelta(days=1)
leave_cancelled_data['LCEQSDiff_QB3'] = (leave_cancelled_data['To Date']  - q3_start) + timedelta(days=1)
leave_cancelled_data['QELCSDiff_QB3'] = (q3_end - leave_cancelled_data['From Date']) + timedelta(days=1)
leave_cancelled_data['LCELCSDiff_QB3'] = (leave_cancelled_data['To Date']  - leave_cancelled_data['From Date']) + timedelta(days=1)

leave_cancelled_data['LCQEQSDiff_QB2'] = (q2_end - q2_start) + timedelta(days=1)
leave_cancelled_data['LCEQSDiff_QB2'] = (leave_cancelled_data['To Date']  - q2_start) + timedelta(days=1)
leave_cancelled_data['QELCSDiff_QB2'] = (q2_end - leave_cancelled_data['From Date']) + timedelta(days=1)
leave_cancelled_data['LCELCSDiff_QB2'] = (leave_cancelled_data['To Date']  - leave_cancelled_data['From Date']) + timedelta(days=1)

leave_cancelled_data['LCQEQSDiff_QB1'] = (q1_end - q1_start) + timedelta(days=1)
leave_cancelled_data['LCEQSDiff_QB1'] = (leave_cancelled_data['To Date']  - q1_start) + timedelta(days=1)
leave_cancelled_data['QELCSDiff_QB1'] = (q1_end - leave_cancelled_data['From Date']) + timedelta(days=1)
leave_cancelled_data['LCELCSDiff_QB1'] = (leave_cancelled_data['To Date']  - leave_cancelled_data['From Date']) + timedelta(days=1)

# calculate the number of days an employee has utilized leaves in each quarter
# leave on 4th quarter
cols_leavecancel_QB4 = ['From Date', 'To Date', 'LCQEQSDiff_QB4',  'LCEQSDiff_QB4', 'QELCSDiff_QB4', 'LCELCSDiff_QB4']
leave_cancelled_data[yr_qtr4] = quarter_days_allocator(leave_cancelled_data, cols_leavecancel_QB4, q4_start, q4_end)

# leave on 3rd quarter
cols_leavecancel_QB3 = ['From Date', 'To Date', 'LCQEQSDiff_QB3',  'LCEQSDiff_QB3', 'QELCSDiff_QB3', 'LCELCSDiff_QB3']
leave_cancelled_data[yr_qtr3] = quarter_days_allocator(leave_cancelled_data, cols_leavecancel_QB3, q3_start, q3_end)

# leave on 2nd quarter
cols_leavecancel_QB2 = ['From Date', 'To Date', 'LCQEQSDiff_QB2',  'LCEQSDiff_QB2', 'QELCSDiff_QB2', 'LCELCSDiff_QB2']
leave_cancelled_data[yr_qtr2] = quarter_days_allocator(leave_cancelled_data, cols_leavecancel_QB2, q2_start, q2_end)

# leave on 1st quarter
cols_leavecancel_QB1 = ['From Date', 'To Date', 'LCQEQSDiff_QB1',  'LCEQSDiff_QB1', 'QELCSDiff_QB1', 'LCELCSDiff_QB1']
leave_cancelled_data[yr_qtr1] = quarter_days_allocator(leave_cancelled_data, cols_leavecancel_QB1, q1_start, q1_end)



# selecting required columns
leave_cancelled_data = leave_cancelled_data[['Masked Code', 'Leave Type', 'TotalLeaveDays',yr_qtr4,yr_qtr3,yr_qtr2,yr_qtr1]]

# converting from days to int
leave_cancelled_data[yr_qtr4] = (leave_cancelled_data[yr_qtr4] / np.timedelta64(1, 'D')).astype(int)
leave_cancelled_data[yr_qtr3] = (leave_cancelled_data[yr_qtr3] / np.timedelta64(1, 'D')).astype(int)
leave_cancelled_data[yr_qtr2] = (leave_cancelled_data[yr_qtr2] / np.timedelta64(1, 'D')).astype(int)
leave_cancelled_data[yr_qtr1] = (leave_cancelled_data[yr_qtr1] / np.timedelta64(1, 'D')).astype(int)

# grouping by employees
leave_cancelled_data = leave_cancelled_data.groupby(['Masked Code','Leave Type'], as_index=False).agg({yr_qtr4:'sum'
                                                                                                  ,yr_qtr3:'sum'                                                                                 
                                                                                                  ,yr_qtr2:'sum'
                                                                                                  ,yr_qtr1:'sum'})

# unpivot the dataset
leave_cancelled_data = leave_cancelled_data.melt(id_vars=['Masked Code','Leave Type'], var_name='YearQuarter', value_name='CancelUtilLeaves')

leave_cancelled_data[['LeaveCode','LeaveCancel']] = leave_cancelled_data['Leave Type'].str.split("-", expand=True) 

# create a key
leave_cancelled_data['Key'] = leave_cancelled_data['Masked Code'] + leave_cancelled_data['LeaveCode'] + leave_cancelled_data['YearQuarter']

leave_cancelled_data = leave_cancelled_data[['Key', 'CancelUtilLeaves']]

leave_approved_data = pd.merge(leave_approved_data, leave_cancelled_data, on='Key', how='left')

leave_approved_data['CancelUtilLeaves'] = leave_approved_data['CancelUtilLeaves'].fillna(0)

del leave_approved_data['Key']

leave_approved_data['UtilLeaves'] = leave_approved_data['RequestUtilLeaves'] - leave_approved_data['CancelUtilLeaves']

leave_approved_data['UtilLeaves'] = np.where(leave_approved_data['UtilLeaves'] < 0, 0, leave_approved_data['UtilLeaves'])


# enrich with leave approved data
# by latest year
leave_approved_data_latestYr = leave_approved_data.copy(deep=True)

# filter the latest year
leave_approved_data_latestYr = leave_approved_data_latestYr[(leave_approved_data_latestYr['YearQuarter'] == yr_qtr1) | 
                                                                  (leave_approved_data_latestYr['YearQuarter'] == yr_qtr2) |
                                                                  (leave_approved_data_latestYr['YearQuarter'] == yr_qtr3) |
                                                                  (leave_approved_data_latestYr['YearQuarter'] == yr_qtr4)]
# subset required columns
leave_approved_data_latestYr = leave_approved_data_latestYr[['Masked Code', 'Leave Type', 'UtilLeaves']]
leave_approved_data_latestYr = leave_approved_data_latestYr.groupby(['Masked Code', 'Leave Type'], as_index=False).agg({ 'UtilLeaves':'sum'})

# create flags
leave_approved_data_latestYr['MLPL_flag'] = np.where(((leave_approved_data_latestYr['Leave Type'] == 'ML-Request')|(leave_approved_data_latestYr['Leave Type'] == 'PL-Request')), 1, 0)
leave_approved_data_latestYr['BL_flag'] = np.where((leave_approved_data_latestYr['Leave Type'] == 'BL-Request'), 1, 0)
leave_approved_data_latestYr['RTL_flag'] = np.where((leave_approved_data_latestYr['Leave Type'] == 'RTL-Request'), 1, 0)
leave_approved_data_latestYr['LWP_leaves'] = np.where((leave_approved_data_latestYr['Leave Type'] == 'LWP-Request'), leave_approved_data_latestYr['UtilLeaves'], 0)
leave_approved_data_latestYr['LatestYr_Leave_Util'] = np.where(
                                                        ((leave_approved_data_latestYr['Leave Type'] == 'LWP-Request')|
                                                         (leave_approved_data_latestYr['Leave Type'] == 'HTO-Request')|
                                                         (leave_approved_data_latestYr['Leave Type'] == 'LE-Request') |
                                                         (leave_approved_data_latestYr['Leave Type'] == 'SPL-Request'))
                                                         ,leave_approved_data_latestYr['UtilLeaves'], 0)

leave_approved_data_latestYr = leave_approved_data_latestYr.groupby(['Masked Code'], as_index=False).agg({ 'MLPL_flag':'sum',
                                                                                                               'BL_flag':'sum',
                                                                                                               'RTL_flag':'sum',
                                                                                                               'LWP_leaves':'sum',
                                                                                                               'LatestYr_Leave_Util':'sum'})
# merge leave utilization data for latest year
DATA = pd.merge(DATA, leave_approved_data_latestYr, on='Masked Code', how='left')

# by latest quarter
# filter the required leave types
leave_approved_data_latestQtr = leave_approved_data[(leave_approved_data['Leave Type'] == 'LE-Request') |
                                                    (leave_approved_data['Leave Type'] == 'HTO-Request')|
                                                    (leave_approved_data['Leave Type'] == 'SPL-Request')|
                                                    (leave_approved_data['Leave Type'] == 'LWP-Request')] 

# filter latest quarter
leave_approved_data_latestQtr = leave_approved_data_latestQtr[(leave_approved_data_latestQtr['YearQuarter'] == yr_qtr1)]

#LWP by quarter
leave_approved_data_latestQtr_LWP = leave_approved_data_latestQtr.copy(deep = True)
leave_approved_data_latestQtr_LWP = leave_approved_data_latestQtr_LWP[(leave_approved_data_latestQtr_LWP['Leave Type'] == 'LWP-Request')]                                                         
leave_approved_data_latestQtr_LWP = leave_approved_data_latestQtr_LWP[['Masked Code', 'UtilLeaves']]
leave_approved_data_latestQtr_LWP.columns = ['Masked Code', 'LatestQtr_LWP_Leave_Util']
DATA = pd.merge(DATA, leave_approved_data_latestQtr_LWP, on='Masked Code', how='left')                                                                              

leave_approved_data_latestQtr = leave_approved_data_latestQtr[['Masked Code', 'UtilLeaves']]
leave_approved_data_latestQtr.columns = ['Masked Code', 'LatestQtr_Leave_Util']
leave_approved_data_latestQtr = leave_approved_data_latestQtr.groupby(['Masked Code'], as_index=False).agg({ 'LatestQtr_Leave_Util':'sum'})
# merge leave utilization data for latest quarter
DATA = pd.merge(DATA, leave_approved_data_latestQtr, on='Masked Code', how='left')

# merge leave request declined data
leave_declined_data= leave_declined_data[(leave_declined_data['YearQuarter'] == yr_qtr1)]
leave_declined_data= leave_declined_data[['Masked Code', 'CountOfRejections']]
DATA = pd.merge(DATA, leave_declined_data, on='Masked Code', how='left')

# fill missing values with 0
DATA['CountOfRejections'] = DATA['CountOfRejections'].fillna(0)

print("Successfully executed Leave Ledger Data\r\n")


Successfully executed Leave Ledger Data



In [41]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Compensation Data\\')

In [42]:

compensation_data = pd.read_excel('Compensation Data.xlsx' , sheet_name ='Sheet1')



# subset required columns
compensation_data = compensation_data[['Encrypted Emp IDs', 'DoJ','Actual Releiving Date',
                                   'Masked FP Latest Year June','Masked FP Latest Year Quarter',
                                   'Masked PP Latest Year']]

compensation_data.columns = ['Masked Code','DoJ','RelievingDate','FP_LatestYr_Jun',
                         'FP_LatestYrQtr','PP_LatestYr']

# converting date to required format
compensation_data['DoJ'] = pd.to_datetime(compensation_data['DoJ'])
compensation_data['RelievingDate'] = pd.to_datetime(compensation_data['RelievingDate'], errors='coerce').dt.date                                             

# calculating yearly increments
compensation_data['Increment_LatestYr'] = np.where(
                                           (compensation_data['FP_LatestYr_Jun'] != 0),
                                                (compensation_data['FP_LatestYrQtr']/compensation_data['FP_LatestYr_Jun']) - 1,
                                                    0)

#logic added for latest run
compensation_data['FP_LatestYr_Jul'] = compensation_data['FP_LatestYrQtr']

# deleting 2017 data
del compensation_data['DoJ']

# enrich with Compensation Data
DATA = pd.merge(DATA, compensation_data, on='Masked Code', how='left')

# calculating latest year percent salary hike
DATA['LatestYrSalaryHike'] =  DATA['Increment_LatestYr'] * 100

print("Successfully executed Compensation Data\r\n")


Successfully executed Compensation Data



In [43]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code\\Input Data\\Oct 2020\\Travel Data\\')

In [44]:

travel_data_domestic = pd.read_excel('TC Air Domestic Data.xlsx', sheet_name = "Sheet1")
travel_data_domestic['ItineraryType'] = 'Domestic'
travel_data_domestic['RefInvoiceNumber'] = np.NaN
travel_data_domestic['SystemInvoiceId'] = np.NaN
travel_data_domestic = travel_data_domestic[['TRIP CODE', 'SystemInvoiceId','RefInvoiceNumber',"Masked Traveller's Emp Id",'ItineraryType','ONEWAY/RETURN','Date of Departure', 'Date of Arrival']]
travel_data_domestic.columns = ['TripReference', 'SystemInvoiceId', 'RefInvoiceNumber', 'Masked Code', 'ItineraryType', 'JourneyType', 'TripStartDate', 'TripEndDate']

travel_data_international = pd.read_excel('TC Air International Data.xlsx', sheet_name = "Sheet1")
travel_data_international['ItineraryType'] = 'International'
travel_data_international['RefInvoiceNumber'] = np.NaN
travel_data_international['SystemInvoiceId'] = np.NaN
travel_data_international = travel_data_international[['TRIP CODE', 'SystemInvoiceId','RefInvoiceNumber',"Masked Traveller's Emp id",'ItineraryType','Journey Type','Departure Date', 'Arrival Date']]
travel_data_international.columns = ['TripReference', 'SystemInvoiceId', 'RefInvoiceNumber', 'Masked Code', 'ItineraryType', 'JourneyType', 'TripStartDate', 'TripEndDate']



# concate all the 3 datasets
travel_data = pd.concat([travel_data_domestic,travel_data_international], axis=0)

# remove trip ref which is 0 or null
travel_data = travel_data[(travel_data['TripReference'].notnull()) | (travel_data['TripReference'] != 0)]

# converting date to required format
travel_data['TripStartDate'] = pd.to_datetime(travel_data['TripStartDate']).dt.date
travel_data['TripEndDate'] = pd.to_datetime(travel_data['TripEndDate']).dt.date

travel_data['TripStartDate'] = pd.to_datetime(travel_data['TripStartDate'])
travel_data['TripEndDate'] = pd.to_datetime(travel_data['TripEndDate'])

# sort the data
travel_data = travel_data.sort_values(by=['Masked Code', 'TripStartDate', 'TripEndDate'], ascending=True)

# create reissued dataset and actual ones
travel_data_reissued = travel_data[['RefInvoiceNumber']]
travel_data_reissued = travel_data_reissued.dropna()
travel_data_reissued['Reissued'] = 1

# removing tickets which has been reissued
travel_data = pd.merge(travel_data, travel_data_reissued, left_on='SystemInvoiceId', right_on='RefInvoiceNumber', how='left')

travel_data = travel_data[travel_data['Reissued'] != 1]

# subset required columns
travel_data = travel_data[['TripReference', 'Masked Code', 'ItineraryType', 'JourneyType', 'TripStartDate', 'TripEndDate']]

# extract year and quarter from the Trip Start Date
travel_data['Quarter'] = pd.PeriodIndex(travel_data['TripStartDate'], freq='Q').strftime('Q%q')
travel_data['Year'] = pd.PeriodIndex(travel_data['TripStartDate'], freq='Y').strftime('%Y')
travel_data['YearQuarter'] = travel_data['Year'] + travel_data['Quarter']

# calculate travel metrics related to travel frequency
travel_data_freq = travel_data[['TripReference', 'Masked Code', 'ItineraryType', 'YearQuarter']]

travel_data_freq = travel_data_freq.groupby(['Masked Code', 'YearQuarter', 'ItineraryType']).TripReference.nunique()

travel_data_freq = pd.DataFrame(travel_data_freq).reset_index()

travel_data_freq['DomesticTravelFreq'] = np.where((travel_data_freq['ItineraryType'] == "Domestic"), travel_data_freq['TripReference'], 0)
travel_data_freq['InternationalTravelFreq'] = np.where((travel_data_freq['ItineraryType'] == "International"), travel_data_freq['TripReference'], 0)

travel_data_freq = travel_data_freq.groupby(['Masked Code', 'YearQuarter'], as_index=False).agg({'DomesticTravelFreq':'sum'
                                                                              ,'InternationalTravelFreq':'sum'})

# calculate total travel frequency
travel_data_freq['TotalTravelFreq'] = travel_data_freq['DomesticTravelFreq'] + travel_data_freq['InternationalTravelFreq']

# create travel flags depending on travel type 
travel_data_freq['IsDomesticTravel'] = np.where((travel_data_freq['DomesticTravelFreq'] >= 1), 1, 0)
travel_data_freq['IsInternationalTravel'] = np.where((travel_data_freq['InternationalTravelFreq'] >= 1), 1, 0)

del travel_data_freq['DomesticTravelFreq']
del travel_data_freq['InternationalTravelFreq']

# calculate trip duration
travel_data_duration = travel_data[['TripReference', 'Masked Code', 'JourneyType', 'TripStartDate', 'TripEndDate']]

travel_data_duration['TripEndDate'] =  travel_data_duration.groupby(['TripReference', 'Masked Code'])['TripEndDate'].transform('max')

travel_data_duration = travel_data_duration.drop_duplicates(subset = ['TripReference', 'Masked Code'], keep = 'first')

travel_data_duration = travel_data_duration.reset_index(drop = True)

travel_data_duration = travel_data_duration[['Masked Code', 'TripStartDate', 'TripEndDate']]


# calculate the date differences (Saturday and Sunday included)
travel_data_duration['TQEQSDiff_QB4'] = (q4_end - q4_start) + timedelta(days=1)
travel_data_duration['TEQSDiff_QB4'] = (travel_data_duration['TripEndDate']  - q4_start) + timedelta(days=1)
travel_data_duration['QETSDiff_QB4'] = (q4_end - travel_data_duration['TripStartDate']) + timedelta(days=1)
travel_data_duration['TETSDiff_QB4'] = (travel_data_duration['TripEndDate']  - travel_data_duration['TripStartDate']) + timedelta(days=1)

travel_data_duration['TQEQSDiff_QB3'] = (q3_end - q3_start) + timedelta(days=1)
travel_data_duration['TEQSDiff_QB3'] = (travel_data_duration['TripEndDate']  - q3_start) + timedelta(days=1)
travel_data_duration['QETSDiff_QB3'] = (q3_end - travel_data_duration['TripStartDate']) + timedelta(days=1)
travel_data_duration['TETSDiff_QB3'] = (travel_data_duration['TripEndDate']  - travel_data_duration['TripStartDate']) + timedelta(days=1)

travel_data_duration['TQEQSDiff_QB2'] = (q2_end - q2_start) + timedelta(days=1)
travel_data_duration['TEQSDiff_QB2'] = (travel_data_duration['TripEndDate']  - q2_start) + timedelta(days=1)
travel_data_duration['QETSDiff_QB2'] = (q2_end - travel_data_duration['TripStartDate']) + timedelta(days=1)
travel_data_duration['TETSDiff_QB2'] = (travel_data_duration['TripEndDate']  - travel_data_duration['TripStartDate']) + timedelta(days=1)

travel_data_duration['TQEQSDiff_QB1'] = (q1_end - q1_start) + timedelta(days=1)
travel_data_duration['TEQSDiff_QB1'] = (travel_data_duration['TripEndDate']  - q1_start) + timedelta(days=1)
travel_data_duration['QETSDiff_QB1'] = (q1_end - travel_data_duration['TripStartDate']) + timedelta(days=1)
travel_data_duration['TETSDiff_QB1'] = (travel_data_duration['TripEndDate']  - travel_data_duration['TripStartDate']) + timedelta(days=1)

# calculate the number of days an employee has travelled in each quarter
# travel on 4th quarter
cols_travel_QB4 = ['TripStartDate', 'TripEndDate', 'TQEQSDiff_QB4',  'TEQSDiff_QB4', 'QETSDiff_QB4', 'TETSDiff_QB4']
travel_data_duration[yr_qtr4] = quarter_days_allocator(travel_data_duration, cols_travel_QB4, q4_start, q4_end)

# travel on 3rd quarter
cols_travel_QB3 = ['TripStartDate', 'TripEndDate', 'TQEQSDiff_QB3',  'TEQSDiff_QB3', 'QETSDiff_QB3', 'TETSDiff_QB3']
travel_data_duration[yr_qtr3] = quarter_days_allocator(travel_data_duration, cols_travel_QB3, q3_start, q3_end)

# travel on 2nd quarter
cols_travel_QB2 = ['TripStartDate', 'TripEndDate', 'TQEQSDiff_QB2',  'TEQSDiff_QB2', 'QETSDiff_QB2', 'TETSDiff_QB2']
travel_data_duration[yr_qtr2] = quarter_days_allocator(travel_data_duration, cols_travel_QB2, q2_start, q2_end)

# travel on 1st quarter
cols_travel_QB1 = ['TripStartDate', 'TripEndDate', 'TQEQSDiff_QB1',  'TEQSDiff_QB1', 'QETSDiff_QB1', 'TETSDiff_QB1']
travel_data_duration[yr_qtr1] = quarter_days_allocator(travel_data_duration, cols_travel_QB1, q1_start, q1_end)



# selecting required columns
travel_data_duration = travel_data_duration[['Masked Code',yr_qtr4,yr_qtr3,yr_qtr2,yr_qtr1]]

# converting from days to int
travel_data_duration[yr_qtr4] = (travel_data_duration[yr_qtr4] / np.timedelta64(1, 'D')).astype(int)
travel_data_duration[yr_qtr3] = (travel_data_duration[yr_qtr3] / np.timedelta64(1, 'D')).astype(int)
travel_data_duration[yr_qtr2] = (travel_data_duration[yr_qtr2] / np.timedelta64(1, 'D')).astype(int)
travel_data_duration[yr_qtr1] = (travel_data_duration[yr_qtr1] / np.timedelta64(1, 'D')).astype(int)

# grouping by employees
travel_data_duration = travel_data_duration.groupby(['Masked Code'], as_index=False).agg({yr_qtr4:'sum'
                                                                                      ,yr_qtr3:'sum'                                                                               
                                                                                      ,yr_qtr2:'sum'
                                                                                      ,yr_qtr1:'sum'})

# unpivot the dataset
travel_data_duration = travel_data_duration.melt(id_vars=['Masked Code'], var_name='YearQuarter', value_name='TravelDuration')


# enrich with travel data
# travel frequency data
# by latest year
travel_data_freq_latestYr = travel_data_freq[(travel_data_freq['YearQuarter'] == yr_qtr1) | 
                                              (travel_data_freq['YearQuarter'] == yr_qtr2) |
                                              (travel_data_freq['YearQuarter'] == yr_qtr3) |
                                              (travel_data_freq['YearQuarter'] == yr_qtr4)]
# subset required columns
travel_data_freq_latestYr = travel_data_freq_latestYr[['Masked Code', 'TotalTravelFreq']]
travel_data_freq_latestYr = travel_data_freq_latestYr.groupby(['Masked Code'], as_index=False).agg({ 'TotalTravelFreq':'sum'})
travel_data_freq_latestYr.columns = ['Masked Code', 'LatestYr_TotalTravelFreq']

# merge with Quarter Block
DATA = pd.merge(DATA, travel_data_freq_latestYr, on='Masked Code', how='left')

# fill missing values with 0
DATA['LatestYr_TotalTravelFreq'] = DATA['LatestYr_TotalTravelFreq'].fillna(0)

# by latest quarter
travel_data_freq_latestQtr = travel_data_freq[(travel_data_freq['YearQuarter'] == yr_qtr1)]

# subset required columns
travel_data_freq_latestQtr = travel_data_freq_latestQtr[['Masked Code', 'TotalTravelFreq', 'IsDomesticTravel', 'IsInternationalTravel']]
travel_data_freq_latestQtr.columns = ['Masked Code', 'LatestQtr_TotalTravelFreq', 'LatestQtr_HasDomesticTravel', 'LatestQtr_HasInternationalTravel']

# merge with Quarter Block
DATA = pd.merge(DATA, travel_data_freq_latestQtr, on='Masked Code', how='left')

# fill missing values with 0
DATA['LatestQtr_TotalTravelFreq'] = DATA['LatestQtr_TotalTravelFreq'].fillna(0)
DATA['LatestQtr_HasDomesticTravel'] = DATA['LatestQtr_HasDomesticTravel'].fillna(0)
DATA['LatestQtr_HasInternationalTravel'] = DATA['LatestQtr_HasInternationalTravel'].fillna(0)

# travel duration data
# by latest year
travel_data_duration_latestYr = travel_data_duration[(travel_data_duration['YearQuarter'] == yr_qtr1) | 
                                                      (travel_data_duration['YearQuarter'] == yr_qtr2) |
                                                      (travel_data_duration['YearQuarter'] == yr_qtr3) |
                                                      (travel_data_duration['YearQuarter'] == yr_qtr4)]
# subset required columns
travel_data_duration_latestYr = travel_data_duration_latestYr[['Masked Code', 'TravelDuration']]
travel_data_duration_latestYr = travel_data_duration_latestYr.groupby(['Masked Code'], as_index=False).agg({'TravelDuration':'sum'})
travel_data_duration_latestYr.columns = ['Masked Code', 'LatestYr_TravelDuration']

# merge with Quarter Block
DATA = pd.merge(DATA, travel_data_duration_latestYr, on='Masked Code', how='left')

# fill missing values with 0
DATA['LatestYr_TravelDuration'] = DATA['LatestYr_TravelDuration'].fillna(0)

# by latest quarter
travel_data_duration_latestQtr = travel_data_duration[(travel_data_duration['YearQuarter'] == yr_qtr1)]

# subset required columns
travel_data_duration_latestQtr = travel_data_duration_latestQtr[['Masked Code', 'TravelDuration']]
travel_data_duration_latestQtr.columns = ['Masked Code', 'LatestQtr_TravelDuration']

# merge with Quarter Block
DATA = pd.merge(DATA, travel_data_duration_latestQtr, on='Masked Code', how='left')

# fill missing values with 0
DATA['LatestQtr_TravelDuration'] = DATA['LatestQtr_TravelDuration'].fillna(0)

print("Successfully executed Travel Data\r\n")


Successfully executed Travel Data



In [45]:
list(DATA.columns)

['Masked Code',
 'DATE_OF_BIRTH',
 'Masked Gender',
 'MARITAL_STATUS',
 'WEDDING_DATE',
 'Masked Location',
 'PERMNANT_STATE',
 'Masked Entity',
 'Masked Status',
 'Masked Department',
 'Masked Function',
 'Masked Grade',
 'NOTICE_PERIOD_IN_DAYS',
 'DATE_OF_JOINING',
 'DATE_OF_LEAVING',
 'Masked RM ID',
 'PARTNER_NAME',
 'REPORTING_TO',
 'EMPLOYEE_STATUS',
 'AGE',
 'YEARS_SINCE_MARRIAGE',
 'YEARS_AT_COMPANY',
 'Office City',
 'Office State',
 'IsSameState',
 'Resignation Date',
 'Attrition',
 'Encrypted Emp ID_x',
 'EmpPerformanceRating',
 'Encrypted Emp ID_y',
 'RMPerformanceRating',
 'EffectiveDate',
 'Grade',
 'IsPromoted',
 'IsPromoted_LatestQtr',
 'IsPromoted_PreviousQtr',
 'LatestYr_BILL_UTIL',
 'LatestYr_PROD_NONBILL_UTIL',
 'LatestYr_NONPROD_NONBILL_UTIL',
 'LatestQtr_BILL_UTIL',
 'LatestQtr_PROD_NONBILL_UTIL',
 'LatestQtr_NONPROD_NONBILL_UTIL',
 'IS_EXPERIENCED',
 'NUM_PRIOR_COMPANIES',
 'PRIOR_EXP',
 'PRIOR_AVG_TENURE',
 'From',
 'From.1',
 'From.2',
 'From.3',
 'From.4',
 'F

In [46]:
yr_qtr1[:4]

'2020'

# <font color='blue'>Attrition Data</font>

In [47]:

# create flag

# DATA['YEAR'] = yr_qtr1[:4]

# DATA['QUARTER'] = yr_qtr1[4:] 

DATA['YEAR'] = '2020'
DATA['QUARTER'] = 'Q2'


# assigning the data
attrition_data= DATA

# calculate YAL and latest grade for each quarter (didn't take into account active and inactive employees)
attrition_data = attrition_data.sort_values(by=['Masked Code', 'YEAR', 'QUARTER'], ascending=True)

# create temp date for YAL creation
attrition_data['YAL_Date'] = np.where((attrition_data['EffectiveDate'].isnull()), attrition_data['DATE_OF_JOINING'], attrition_data['EffectiveDate'])

# calculate YAL
attrition_data['Days_at_Level'] = q1_end - attrition_data['YAL_Date']


attrition_data['YAL'] = attrition_data['Days_at_Level']/np.timedelta64(1,'Y')

# calculate latest grade for quarter
attrition_data['LatestGrade'] = np.where((attrition_data['Grade'].isnull()), attrition_data['Masked Grade'], attrition_data['Grade'])

# promoted flag
attrition_data['IsPromoted'] = np.where((attrition_data['IsPromoted'] == 'Promoted'), 1 ,0)

# capping project hours for snapshot data
attrition_data['LatestQtrProjectDays'] = np.where((attrition_data['LatestQtrProjectDays'] > 91), 91, attrition_data['LatestQtrProjectDays'])
attrition_data['LatestYrProjectDays'] = np.where((attrition_data['LatestYrProjectDays'] > 365), 365, attrition_data['LatestYrProjectDays'])

# imputing outliers with median values for compensation data
attrition_data.loc[attrition_data['LatestYrSalaryHike'] > 100 , 'LatestYrSalaryHike'] = attrition_data['LatestYrSalaryHike'].median()

#  capping salary hike which are negative
attrition_data['LatestYrSalaryHike'] = np.where((attrition_data['LatestYrSalaryHike'] < 0),
                                                0, attrition_data['LatestYrSalaryHike'])

# imputing nulls with 0 for LatestYrSalaryHike
attrition_data['LatestYrSalaryHike'] = np.where((attrition_data['LatestYrSalaryHike'].isnull()),
                                                0, attrition_data['LatestYrSalaryHike'])

# replacing staff util negative values with 0
attrition_data['LatestYr_BILL_UTIL'][attrition_data['LatestYr_BILL_UTIL'] < 0] = 0
attrition_data['LatestYr_PROD_NONBILL_UTIL'][attrition_data['LatestYr_PROD_NONBILL_UTIL'] < 0] = 0
attrition_data['LatestYr_NONPROD_NONBILL_UTIL'][attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] < 0] = 0

attrition_data['LatestQtr_BILL_UTIL'][attrition_data['LatestQtr_BILL_UTIL'] < 0] = 0
attrition_data['LatestQtr_PROD_NONBILL_UTIL'][attrition_data['LatestQtr_PROD_NONBILL_UTIL'] < 0] = 0
attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'][attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] < 0] = 0

# replace inf with 0
attrition_data['LatestYr_BILL_UTIL'] = attrition_data['LatestYr_BILL_UTIL'].replace(np.inf, 0)
attrition_data['LatestYr_PROD_NONBILL_UTIL'] = attrition_data['LatestYr_PROD_NONBILL_UTIL'].replace(np.inf, 0)
attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestYr_NONPROD_NONBILL_UTIL'].replace(np.inf, 0)
attrition_data['LatestQtr_BILL_UTIL'] = attrition_data['LatestQtr_BILL_UTIL'].replace(np.inf, 0)
attrition_data['LatestQtr_PROD_NONBILL_UTIL'] = attrition_data['LatestQtr_PROD_NONBILL_UTIL'].replace(np.inf, 0)
attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'].replace(np.inf, 0)

#impute null values with 0
attrition_data['LatestYr_BILL_UTIL'] = attrition_data['LatestYr_BILL_UTIL'].fillna(0)
attrition_data['LatestYr_PROD_NONBILL_UTIL'] = attrition_data['LatestYr_PROD_NONBILL_UTIL'].fillna(0)
attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestYr_NONPROD_NONBILL_UTIL'].fillna(0)

attrition_data['LatestQtr_BILL_UTIL'] = attrition_data['LatestQtr_BILL_UTIL'].fillna(0)
attrition_data['LatestQtr_PROD_NONBILL_UTIL'] = attrition_data['LatestQtr_PROD_NONBILL_UTIL'].fillna(0)
attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'].fillna(0)

# calculating percentile
# staff Utilization by latest YEAR
attrition_data['LatestYr_PER_BILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER', 'Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_BILL_UTIL'].rank(pct=True, method = 'min')
attrition_data['LatestYr_PER_PROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_PROD_NONBILL_UTIL'].rank(pct=True, method = 'min')
attrition_data['LatestYr_PER_NONPROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_NONPROD_NONBILL_UTIL'].rank(pct=True, method = 'min')

# staff Utilization by latest QUARTER
attrition_data['LatestQtr_PER_BILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_BILL_UTIL'].rank(pct=True, method = 'min')
attrition_data['LatestQtr_PER_PROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_PROD_NONBILL_UTIL'].rank(pct=True, method = 'min')
attrition_data['LatestQtr_PER_NONPROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_NONPROD_NONBILL_UTIL'].rank(pct=True, method = 'min')

#imputing null values of PP salary with 0
attrition_data['PP_LatestYr']  = np.where((attrition_data['PP_LatestYr'].isnull()), 0, attrition_data['PP_LatestYr'])

#salary variables
attrition_data['FP_SALARY'] = attrition_data['FP_LatestYr_Jul']
attrition_data['PP_SALARY'] =  attrition_data['PP_LatestYr']

# percentile salary
attrition_data['FP_PERCENTILE_SALARY'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['FP_LatestYr_Jul'].rank(pct=True, method = 'min')
attrition_data['PP_PERCENTILE_SALARY'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['PP_LatestYr'].rank(pct=True, method = 'min')

# calculate percentile for travel frequency
attrition_data['PER_LatestYr_TotalTravelFreq'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_TotalTravelFreq'].rank(pct=True, method = 'min')

# calculate percentile for travel frequency
attrition_data['PER_LatestQtr_TotalTravelFreq'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_TotalTravelFreq'].rank(pct=True, method = 'min')

# calculate percentile for travel duration
attrition_data['PER_LatestQtr_TravelDuration'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_TravelDuration'].rank(pct=True, method = 'min')

# calculate percentile for travel duration
attrition_data['PER_LatestYr_TravelDuration'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_TravelDuration'].rank(pct=True, method = 'min')

#YearQuarter
attrition_data['YearQuarter'] = attrition_data['YEAR'] + attrition_data['QUARTER']

# Leave utilised Year Quarter ratio
attrition_data['Leave_Util_LatestQtrYr_Ratio'] = attrition_data['LatestQtr_Leave_Util']/attrition_data['LatestYr_Leave_Util']

#  capping utilisation which are greater than 2
attrition_data['LatestYr_BILL_UTIL'] = np.where((attrition_data['LatestYr_BILL_UTIL'] > 2), 2, attrition_data['LatestYr_BILL_UTIL'])
attrition_data['LatestYr_PROD_NONBILL_UTIL'] = np.where((attrition_data['LatestYr_PROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestYr_PROD_NONBILL_UTIL'])
attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] = np.where((attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestYr_NONPROD_NONBILL_UTIL'])

attrition_data['LatestQtr_BILL_UTIL'] = np.where((attrition_data['LatestQtr_BILL_UTIL'] > 2), 2, attrition_data['LatestQtr_BILL_UTIL'])
attrition_data['LatestQtr_PROD_NONBILL_UTIL'] = np.where((attrition_data['LatestQtr_PROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestQtr_PROD_NONBILL_UTIL'])
attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] = np.where((attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'])

print("Successfully handled Outliers\r\n")


Successfully handled Outliers



In [48]:

# # Appending all the quarter blocks
# # create flags for each quarter blocks
# DATA['YEAR'] = '2018'
# DATA['QUARTER'] = 'Q3'

# DATA['YEAR'] = '2018'
# DATA['QUARTER'] = 'Q4'

# DATA['YEAR'] = '2019'
# DATA['QUARTER'] = 'Q1'

# DATA['YEAR'] = '2019'
# DATA['QUARTER'] = 'Q2'

# DATA['YEAR'] = '2019'
# DATA['QUARTER'] = 'Q3'

# DATA['YEAR'] = '2019'
# DATA['QUARTER'] = 'Q4'

# attrition_data = DATA


# # attrition_data = pd.concat([QUARTER_BLOCK1, QUARTER_BLOCK2, QUARTER_BLOCK3, QUARTER_BLOCK4, QUARTER_BLOCK5, QUARTER_BLOCK6], axis=0)
# # create flag
# # DATA['YEAR'] = yr_qtr1[:4]

# # DATA['QUARTER'] = yr_qtr1[4:] 

# # assigning the data

# # calculate YAL and latest grade for each quarter (didn't take into account active and inactive employees)
# attrition_data = attrition_data.sort_values(by=['Masked Code', 'YEAR', 'QUARTER'], ascending=True)

# # create temp date for YAL creation
# attrition_data['YAL_Date'] = np.where((attrition_data['EffectiveDate'].isnull()), attrition_data['DATE_OF_JOINING'], attrition_data['EffectiveDate'])

# # calculate YAL
# attrition_data['Days_at_Level'] = np.where(((attrition_data['YEAR'] == '2018') & (attrition_data['QUARTER'] == 'Q3')), 
#                                         (q1_end - attrition_data['YAL_Date']), 
#                                            np.where(((attrition_data['YEAR'] == '2018') & (attrition_data['QUARTER'] == 'Q4')),
#                                                    (q2_end - attrition_data['YAL_Date']), 
#                                                        np.where(((attrition_data['YEAR'] == '2019') & (attrition_data['QUARTER'] == 'Q1')),
#                                                            (q3_end - attrition_data['YAL_Date']), 
#                                                                 np.where(((attrition_data['YEAR'] == '2019') & (attrition_data['QUARTER'] == 'Q2')),
#                                                                    (q4_end - attrition_data['YAL_Date']), 
#                                                                           np.where(((attrition_data['YEAR'] == '2019') & (attrition_data['QUARTER'] == 'Q3')),
#                                                                                (q5_end - attrition_data['YAL_Date']), 
#                                                                                     np.where(((attrition_data['YEAR'] == '2019') & (attrition_data['QUARTER'] == 'Q4')),
#                                                                                            (q6_end - attrition_data['YAL_Date']), 
#                                                                                                  0
#                                                                                                     ))))))     


# attrition_data['YAL'] = attrition_data['Days_at_Level']/np.timedelta64(1,'Y')

# # calculate latest grade for quarter
# attrition_data['LatestGrade'] = np.where((attrition_data['Grade'].isnull()), attrition_data['Masked Grade'], attrition_data['Grade'])

# # promoted flag
# attrition_data['IsPromoted'] = np.where((attrition_data['IsPromoted'] == 'Promoted'), 1 ,0)

# # capping project hours for snapshot data
# attrition_data['LatestQtrProjectDays'] = np.where((attrition_data['LatestQtrProjectDays'] > 91), 91, attrition_data['LatestQtrProjectDays'])
# attrition_data['LatestYrProjectDays'] = np.where((attrition_data['LatestYrProjectDays'] > 365), 365, attrition_data['LatestYrProjectDays'])

# # imputing outliers with median values for compensation data
# attrition_data.loc[attrition_data['LatestYrSalaryHike'] > 100 , 'LatestYrSalaryHike'] = attrition_data['LatestYrSalaryHike'].median()

# #  capping salary hike which are negative
# attrition_data['LatestYrSalaryHike'] = np.where((attrition_data['LatestYrSalaryHike'] < 0),
#                                                 0, attrition_data['LatestYrSalaryHike'])

# # imputing nulls with 0 for LatestYrSalaryHike
# attrition_data['LatestYrSalaryHike'] = np.where((attrition_data['LatestYrSalaryHike'].isnull()),
#                                                 0, attrition_data['LatestYrSalaryHike'])

# # replacing staff util negative values with 0
# attrition_data['LatestYr_BILL_UTIL'][attrition_data['LatestYr_BILL_UTIL'] < 0] = 0
# attrition_data['LatestYr_PROD_NONBILL_UTIL'][attrition_data['LatestYr_PROD_NONBILL_UTIL'] < 0] = 0
# attrition_data['LatestYr_NONPROD_NONBILL_UTIL'][attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] < 0] = 0

# attrition_data['LatestQtr_BILL_UTIL'][attrition_data['LatestQtr_BILL_UTIL'] < 0] = 0
# attrition_data['LatestQtr_PROD_NONBILL_UTIL'][attrition_data['LatestQtr_PROD_NONBILL_UTIL'] < 0] = 0
# attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'][attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] < 0] = 0

# # replace inf with 0
# attrition_data['LatestYr_BILL_UTIL'] = attrition_data['LatestYr_BILL_UTIL'].replace(np.inf, 0)
# attrition_data['LatestYr_PROD_NONBILL_UTIL'] = attrition_data['LatestYr_PROD_NONBILL_UTIL'].replace(np.inf, 0)
# attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestYr_NONPROD_NONBILL_UTIL'].replace(np.inf, 0)
# attrition_data['LatestQtr_BILL_UTIL'] = attrition_data['LatestQtr_BILL_UTIL'].replace(np.inf, 0)
# attrition_data['LatestQtr_PROD_NONBILL_UTIL'] = attrition_data['LatestQtr_PROD_NONBILL_UTIL'].replace(np.inf, 0)
# attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'].replace(np.inf, 0)

# #impute null values with 0
# attrition_data['LatestYr_BILL_UTIL'] = attrition_data['LatestYr_BILL_UTIL'].fillna(0)
# attrition_data['LatestYr_PROD_NONBILL_UTIL'] = attrition_data['LatestYr_PROD_NONBILL_UTIL'].fillna(0)
# attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestYr_NONPROD_NONBILL_UTIL'].fillna(0)

# attrition_data['LatestQtr_BILL_UTIL'] = attrition_data['LatestQtr_BILL_UTIL'].fillna(0)
# attrition_data['LatestQtr_PROD_NONBILL_UTIL'] = attrition_data['LatestQtr_PROD_NONBILL_UTIL'].fillna(0)
# attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] = attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'].fillna(0)

# # calculating percentile
# # staff Utilization by latest YEAR
# attrition_data['LatestYr_PER_BILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER', 'Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_BILL_UTIL'].rank(pct=True, method = 'min')
# attrition_data['LatestYr_PER_PROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_PROD_NONBILL_UTIL'].rank(pct=True, method = 'min')
# attrition_data['LatestYr_PER_NONPROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_NONPROD_NONBILL_UTIL'].rank(pct=True, method = 'min')

# # staff Utilization by latest QUARTER
# attrition_data['LatestQtr_PER_BILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_BILL_UTIL'].rank(pct=True, method = 'min')
# attrition_data['LatestQtr_PER_PROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_PROD_NONBILL_UTIL'].rank(pct=True, method = 'min')
# attrition_data['LatestQtr_PER_NONPROD_NONBILL_UTIL'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_NONPROD_NONBILL_UTIL'].rank(pct=True, method = 'min')

# #imputing null values of PP salary with 0
# attrition_data['PP_LatestYr']  = np.where((attrition_data['PP_LatestYr'].isnull()), 0, attrition_data['PP_LatestYr'])

# #salary variables
# attrition_data['FP_SALARY'] = attrition_data['FP_LatestYr_Jul']
# attrition_data['PP_SALARY'] =  attrition_data['PP_LatestYr']

# # percentile salary
# attrition_data['FP_PERCENTILE_SALARY'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['FP_LatestYr_Jul'].rank(pct=True, method = 'min')
# attrition_data['PP_PERCENTILE_SALARY'] = attrition_data.groupby(['YEAR', 'QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['PP_LatestYr'].rank(pct=True, method = 'min')

# # calculate percentile for travel frequency
# attrition_data['PER_LatestYr_TotalTravelFreq'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_TotalTravelFreq'].rank(pct=True, method = 'min')

# # calculate percentile for travel frequency
# attrition_data['PER_LatestQtr_TotalTravelFreq'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_TotalTravelFreq'].rank(pct=True, method = 'min')

# # calculate percentile for travel duration
# attrition_data['PER_LatestQtr_TravelDuration'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestQtr_TravelDuration'].rank(pct=True, method = 'min')

# # calculate percentile for travel duration
# attrition_data['PER_LatestYr_TravelDuration'] = attrition_data.groupby(['YEAR','QUARTER','Masked Function', 'Masked Department', 'LatestGrade'])['LatestYr_TravelDuration'].rank(pct=True, method = 'min')

# #YearQuarter
# attrition_data['YearQuarter'] = attrition_data['YEAR'] + attrition_data['QUARTER']

# # Leave utilised Year Quarter ratio
# attrition_data['Leave_Util_LatestQtrYr_Ratio'] = attrition_data['LatestQtr_Leave_Util']/attrition_data['LatestYr_Leave_Util']

# #  capping utilisation which are greater than 2
# attrition_data['LatestYr_BILL_UTIL'] = np.where((attrition_data['LatestYr_BILL_UTIL'] > 2), 2, attrition_data['LatestYr_BILL_UTIL'])
# attrition_data['LatestYr_PROD_NONBILL_UTIL'] = np.where((attrition_data['LatestYr_PROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestYr_PROD_NONBILL_UTIL'])
# attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] = np.where((attrition_data['LatestYr_NONPROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestYr_NONPROD_NONBILL_UTIL'])

# attrition_data['LatestQtr_BILL_UTIL'] = np.where((attrition_data['LatestQtr_BILL_UTIL'] > 2), 2, attrition_data['LatestQtr_BILL_UTIL'])
# attrition_data['LatestQtr_PROD_NONBILL_UTIL'] = np.where((attrition_data['LatestQtr_PROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestQtr_PROD_NONBILL_UTIL'])
# attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] = np.where((attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'] > 2), 2, attrition_data['LatestQtr_NONPROD_NONBILL_UTIL'])

# print("Successfully handled Outliers\r\n")


### Missing Values Treatment

In [49]:

attrition_data['YEARS_SINCE_MARRIAGE']  = np.where(
                                                        ((attrition_data['MARITAL_STATUS'] == 'Single')|
                                                         (attrition_data['MARITAL_STATUS'] == 'Widow') |
                                                         (attrition_data['MARITAL_STATUS'] == 'Divorced')), 
                                                                0, 
                                                                    attrition_data['YEARS_SINCE_MARRIAGE'])                                                      

attrition_data['MARITAL_STATUS'] = np.where((attrition_data['MARITAL_STATUS'].isnull()), 'Not Available', attrition_data['MARITAL_STATUS'])

attrition_data['YEARS_SINCE_MARRIAGE'] = np.where((attrition_data['YEARS_SINCE_MARRIAGE'] >= 0), attrition_data['YEARS_SINCE_MARRIAGE'], 0)

attrition_data['IsPromoted']  = np.where((attrition_data['IsPromoted'].isnull()), 'Non-Promoted', attrition_data['IsPromoted'])

attrition_data['TOTAL_WORK_EXP'] = np.where((attrition_data['IS_EXPERIENCED'] == 0), attrition_data['YEARS_AT_COMPANY'], attrition_data['TOTAL_WORK_EXP'])

attrition_data['PRIOR_EXP'] = np.where((attrition_data['IS_EXPERIENCED'] == 0), 0, attrition_data['PRIOR_EXP'])

attrition_data['PRIOR_AVG_TENURE'] = np.where((attrition_data['IS_EXPERIENCED'] == 0), 0, attrition_data['PRIOR_AVG_TENURE'])

attrition_data['NUM_PRIOR_COMPANIES'] = np.where((attrition_data['IS_EXPERIENCED'] == 0), 0, attrition_data['NUM_PRIOR_COMPANIES'])

attrition_data['LatestQtrProjectDays'] = np.where((attrition_data['LatestQtrProjectDays'].isnull()), 0, attrition_data['LatestQtrProjectDays'])

attrition_data['LatestYrProjectDays'] = np.where((attrition_data['LatestYrProjectDays'].isnull()), 0, attrition_data['LatestYrProjectDays'])

attrition_data['LatestQtr_Leave_Util'] = np.where((attrition_data['LatestQtr_Leave_Util'].isnull()), 0, attrition_data['LatestQtr_Leave_Util'])

attrition_data['LatestYr_Leave_Util'] = np.where((attrition_data['LatestYr_Leave_Util'].isnull()), 0, attrition_data['LatestYr_Leave_Util'])

attrition_data['MLPL_flag'] = np.where((attrition_data['MLPL_flag'].isnull()), 0, attrition_data['MLPL_flag'])

attrition_data['BL_flag'] = np.where((attrition_data['BL_flag'].isnull()), 0, attrition_data['BL_flag'])

attrition_data['RTL_flag'] = np.where((attrition_data['RTL_flag'].isnull()), 0, attrition_data['RTL_flag'])

attrition_data['LatestYr_LWP_Leave_Util'] = np.where((attrition_data['LWP_leaves'].isnull()), 0, attrition_data['LWP_leaves'])

attrition_data['LatestQtr_snapshot_composite_score'] = np.where((attrition_data['LatestQtr_snapshot_composite_score'].isnull()), 0, attrition_data['LatestQtr_snapshot_composite_score'])

attrition_data['LatestYr_snapshot_composite_score'] = np.where((attrition_data['LatestYr_snapshot_composite_score'].isnull()), 0, attrition_data['LatestYr_snapshot_composite_score'])

attrition_data['IsPromoted_LatestQtr'] =  np.where((attrition_data['IsPromoted_LatestQtr'].isnull()), 0, attrition_data['IsPromoted_LatestQtr'])

attrition_data['IsPromoted_PreviousQtr'] =  np.where((attrition_data['IsPromoted_PreviousQtr'].isnull()), 0, attrition_data['IsPromoted_PreviousQtr'])

attrition_data['LatestQtr_LWP_Leave_Util'] = np.where((attrition_data['LatestQtr_LWP_Leave_Util'].isnull()), 0, attrition_data['LatestQtr_LWP_Leave_Util'])

attrition_data['EmpPerformanceRating'] = np.where((attrition_data['EmpPerformanceRating'].isnull()), 'Not Eligible', attrition_data['EmpPerformanceRating'])

attrition_data['RMPerformanceRating'] = np.where((attrition_data['RMPerformanceRating'].isnull()), 'Not Eligible', attrition_data['RMPerformanceRating'])

attrition_data['Leave_Util_LatestQtrYr_Ratio'] =np.where((attrition_data['Leave_Util_LatestQtrYr_Ratio'].isnull()), 0, attrition_data['Leave_Util_LatestQtrYr_Ratio'])

print("Successfully handled Missing Values\r\n")


Successfully handled Missing Values



In [50]:

# Subsetting final data
attrition_data = attrition_data[['Masked Code', 'YEAR', 'QUARTER','YearQuarter','Masked Gender', 'MARITAL_STATUS','Masked Entity','Masked Status', 'Masked Department', 'DATE_OF_JOINING','Masked RM ID',
                            'Masked Function','PARTNER_NAME','REPORTING_TO', 'LatestGrade', 'NOTICE_PERIOD_IN_DAYS','AGE','YEARS_SINCE_MARRIAGE', 
                            'Masked Location', 'IsSameState','YEARS_AT_COMPANY', 'IsPromoted','IsPromoted_LatestQtr','IsPromoted_PreviousQtr','YAL',
                            'LatestYr_BILL_UTIL', 'LatestYr_PROD_NONBILL_UTIL', 'LatestYr_NONPROD_NONBILL_UTIL',
                            'LatestQtr_BILL_UTIL', 'LatestQtr_PROD_NONBILL_UTIL', 'LatestQtr_NONPROD_NONBILL_UTIL',
                            'LatestYr_PER_BILL_UTIL', 'LatestYr_PER_PROD_NONBILL_UTIL', 'LatestYr_PER_NONPROD_NONBILL_UTIL',
                            'LatestQtr_PER_BILL_UTIL', 'LatestQtr_PER_PROD_NONBILL_UTIL', 'LatestQtr_PER_NONPROD_NONBILL_UTIL',
                            'IS_EXPERIENCED','TOTAL_WORK_EXP','PRIOR_EXP','PRIOR_AVG_TENURE','NUM_PRIOR_COMPANIES',
                            'LAST5YRS_COMP_WORKED','LatestQtrTechLearnHours','LatestQtrProfLeadDevLearnHours','LatestYrTechLearnHours','LatestYrProfLeadDevLearnHours',
                            'LatestQtrTechNumCourses','LatestQtrProfLeadDevNumCourses','LatestYrTechNumCourses','LatestYrProfLeadDevNumCourses',
                            'NumOfRewards','NumOfRecognitions','TotalRnR','IsRewarded','IsRecognized','Amount(Rs)','LatestQtrProjectDays',
                            'LatestYrProjectDays','LatestQtrPerfSnapshot','LatestQtrOtherSnapshot','LatestQtrTotalSnapshot',
                            'LatestYrPerfSnapshot','LatestYrOtherSnapshot','LatestYrTotalSnapshot','LatestQtr_snapshot_composite_score','LatestYr_snapshot_composite_score','LatestQtr_Leave_Util',
                            'LatestYr_Leave_Util','Leave_Util_LatestQtrYr_Ratio','MLPL_flag','BL_flag','RTL_flag','LatestYr_LWP_Leave_Util','LatestQtr_LWP_Leave_Util','CountOfRejections','FP_SALARY','PP_SALARY','FP_PERCENTILE_SALARY',
                            'PP_PERCENTILE_SALARY','LatestYrSalaryHike','EmpPerformanceRating','RMPerformanceRating',
                            'LatestQtr_HasDomesticTravel', 'LatestQtr_HasInternationalTravel', 'LatestQtr_TotalTravelFreq', 'LatestQtr_TravelDuration','LatestYr_TotalTravelFreq', 'LatestYr_TravelDuration',
                            'PER_LatestQtr_TotalTravelFreq', 'PER_LatestQtr_TravelDuration',
                            'PER_LatestYr_TotalTravelFreq', 'PER_LatestYr_TravelDuration', 'Attrition']]

# rename the columns
attrition_data = attrition_data.rename(columns={"Masked Code": "Masked_Code",
                                                        "Masked Gender": "Masked_Gender",
                                                        "Masked Status": "Masked_Status",
                                                        "Masked Entity": "Masked_Entity",
                                                        "Masked Function": "Masked_Function",
                                                        "Masked Location": "Masked_Location",
                                                        "Masked RM ID" : "Masked_RM_ID",
                                                        "Amount(Rs)": "RewardAmount"})


print("Successfully included all the variables into final data\r\n")


Successfully included all the variables into final data



In [51]:
len(DATA.loc[DATA['Attrition']==1])

138

In [52]:
list(attrition_data.columns)

['Masked_Code',
 'YEAR',
 'QUARTER',
 'YearQuarter',
 'Masked_Gender',
 'MARITAL_STATUS',
 'Masked_Entity',
 'Masked_Status',
 'Masked Department',
 'DATE_OF_JOINING',
 'Masked_RM_ID',
 'Masked_Function',
 'PARTNER_NAME',
 'REPORTING_TO',
 'LatestGrade',
 'NOTICE_PERIOD_IN_DAYS',
 'AGE',
 'YEARS_SINCE_MARRIAGE',
 'Masked_Location',
 'IsSameState',
 'YEARS_AT_COMPANY',
 'IsPromoted',
 'IsPromoted_LatestQtr',
 'IsPromoted_PreviousQtr',
 'YAL',
 'LatestYr_BILL_UTIL',
 'LatestYr_PROD_NONBILL_UTIL',
 'LatestYr_NONPROD_NONBILL_UTIL',
 'LatestQtr_BILL_UTIL',
 'LatestQtr_PROD_NONBILL_UTIL',
 'LatestQtr_NONPROD_NONBILL_UTIL',
 'LatestYr_PER_BILL_UTIL',
 'LatestYr_PER_PROD_NONBILL_UTIL',
 'LatestYr_PER_NONPROD_NONBILL_UTIL',
 'LatestQtr_PER_BILL_UTIL',
 'LatestQtr_PER_PROD_NONBILL_UTIL',
 'LatestQtr_PER_NONPROD_NONBILL_UTIL',
 'IS_EXPERIENCED',
 'TOTAL_WORK_EXP',
 'PRIOR_EXP',
 'PRIOR_AVG_TENURE',
 'NUM_PRIOR_COMPANIES',
 'LAST5YRS_COMP_WORKED',
 'LatestQtrTechLearnHours',
 'LatestQtrProfLeadDev

In [53]:
# attrition_data.drop(['Masked_RM_ID','PARTNER_NAME','REPORTING_TO'], axis = 1) 
attrition_data = attrition_data[['Masked_Code'
,'YEAR','Masked_RM_ID','PARTNER_NAME','REPORTING_TO'
,'QUARTER'
,'YearQuarter'
,'Masked_Gender'
,'MARITAL_STATUS'
,'Masked_Entity'
,'Masked_Status'
,'Masked Department'
,'DATE_OF_JOINING'
,'Masked_Function'
,'LatestGrade'
,'NOTICE_PERIOD_IN_DAYS'
,'AGE'
,'YEARS_SINCE_MARRIAGE'
,'Masked_Location'
,'IsSameState'
,'YEARS_AT_COMPANY'
,'IsPromoted'
,'IsPromoted_LatestQtr'
,'IsPromoted_PreviousQtr'
,'YAL'
,'LatestYr_BILL_UTIL'
,'LatestYr_PROD_NONBILL_UTIL'
,'LatestYr_NONPROD_NONBILL_UTIL'
,'LatestQtr_BILL_UTIL'
,'LatestQtr_PROD_NONBILL_UTIL'
,'LatestQtr_NONPROD_NONBILL_UTIL'
,'LatestYr_PER_BILL_UTIL'
,'LatestYr_PER_PROD_NONBILL_UTIL'
,'LatestYr_PER_NONPROD_NONBILL_UTIL'
,'LatestQtr_PER_BILL_UTIL'
,'LatestQtr_PER_PROD_NONBILL_UTIL'
,'LatestQtr_PER_NONPROD_NONBILL_UTIL'
,'IS_EXPERIENCED'
,'TOTAL_WORK_EXP'
,'PRIOR_EXP'
,'PRIOR_AVG_TENURE'
,'NUM_PRIOR_COMPANIES'
,'LAST5YRS_COMP_WORKED'
,'LatestQtrTechLearnHours'
,'LatestQtrProfLeadDevLearnHours'
,'LatestYrTechLearnHours'
,'LatestYrProfLeadDevLearnHours'
,'LatestQtrTechNumCourses'
,'LatestQtrProfLeadDevNumCourses'
,'LatestYrTechNumCourses'
,'LatestYrProfLeadDevNumCourses'
,'NumOfRewards'
,'NumOfRecognitions'
,'TotalRnR'
,'IsRewarded'
,'IsRecognized'
,'RewardAmount'
,'LatestQtrProjectDays'
,'LatestYrProjectDays'
,'LatestQtrPerfSnapshot'
,'LatestQtrOtherSnapshot'
,'LatestQtrTotalSnapshot'
,'LatestYrPerfSnapshot'
,'LatestYrOtherSnapshot'
,'LatestYrTotalSnapshot'
,'LatestQtr_snapshot_composite_score'
,'LatestYr_snapshot_composite_score'
,'LatestQtr_Leave_Util'
,'LatestYr_Leave_Util'
,'Leave_Util_LatestQtrYr_Ratio'
,'MLPL_flag'
,'BL_flag'
,'RTL_flag'
,'LatestYr_LWP_Leave_Util'
,'LatestQtr_LWP_Leave_Util'
,'CountOfRejections'
,'FP_SALARY'
,'PP_SALARY'
,'FP_PERCENTILE_SALARY'
,'PP_PERCENTILE_SALARY'
,'LatestYrSalaryHike'
,'EmpPerformanceRating'
,'RMPerformanceRating'
,'LatestQtr_HasDomesticTravel'
,'LatestQtr_HasInternationalTravel'
,'LatestQtr_TotalTravelFreq'
,'LatestQtr_TravelDuration'
,'LatestYr_TotalTravelFreq'
,'LatestYr_TravelDuration'
,'PER_LatestQtr_TotalTravelFreq'
,'PER_LatestQtr_TravelDuration'
,'PER_LatestYr_TotalTravelFreq'
,'PER_LatestYr_TravelDuration'
,'Attrition'
]]

In [54]:
attrition_data_F002 = attrition_data[attrition_data['Masked_Function']== "F002"]
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code/Code/Mapping Table/')
# get the department group for F002
F002_Department_Group = pd.read_excel('F002 Department Group.xlsx', sheet_name = "Sheet1")
attrition_data_F002 = pd.merge(attrition_data_F002, F002_Department_Group, on='Masked Department', how='left')


# get the department group for F005

#dividing data based on function
attrition_data_F005 = attrition_data[attrition_data['Masked_Function']== "F005"]
attrition_data_F005['Masked_Department_Group'] = np.where((attrition_data_F005['Masked Department'] == "SL080"), 
                                                                    "SL052", 
                                                                            attrition_data_F005['Masked Department'])

In [55]:
# list(attrition_data_F002['Masked Department'].unique())

In [56]:
#edit
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\')
writer = pd.ExcelWriter('Feature_Universe_2020Q2_F002.xlsx', engine='xlsxwriter', datetime_format='yyyy-mm-dd')
attrition_data_F002.to_excel(writer, index= False)
writer.save()

os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\')
writer = pd.ExcelWriter('Feature_Universe_2020Q2_F005.xlsx', engine='xlsxwriter', datetime_format='yyyy-mm-dd')
attrition_data_F005.to_excel(writer, index= False)
writer.save()

In [57]:
print'dropoff

SyntaxError: EOL while scanning string literal (<ipython-input-57-9a4cc416d1ea>, line 1)

In [ ]:

# ## converting F005 few variables to percentiles
# attrition_data['PER_LatestQtrProjectDays'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestQtrProjectDays'].rank(pct=True, method = 'min')
# attrition_data['PER_YAL'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['YAL'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestQtr_snapshot_composite_score'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestQtr_snapshot_composite_score'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestQtrProfLeadDevLearnHours'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestQtrProfLeadDevLearnHours'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestQtrProfLeadDevNumCourses'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestQtrProfLeadDevNumCourses'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestQtrPerfSnapshot'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestQtrPerfSnapshot'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestQtrTechLearnHours'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestQtrTechLearnHours'].rank(pct=True, method = 'min')
# attrition_data['PER_Leave_Util_LatestQtrYr_Ratio'] = attrition_data.groupby(['YEAR','Masked_Function', 'Masked Department', 'LatestGrade'])['Leave_Util_LatestQtrYr_Ratio'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestYrPerfSnapshot'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestYrPerfSnapshot'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestYr_snapshot_composite_score'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestYr_snapshot_composite_score'].rank(pct=True, method = 'min')
# attrition_data['PER_LatestYrProjectDays'] = attrition_data.groupby(['YEAR','QUARTER','Masked_Function', 'Masked Department', 'LatestGrade'])['LatestYrProjectDays'].rank(pct=True, method = 'min')

# print("Successfully converted variables into percentile required for Model\r\n")


In [ ]:
print(yr_qtr1[4:])
DATA['YEAR'].unique()

In [ ]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code/Code/Mapping Table/')

### Data for F002 model

In [ ]:
# dividing data based on function
attrition_data_F002 = attrition_data[attrition_data['Masked_Function']== "F002"]

# get the department group for F002
F002_Department_Group = pd.read_excel('F002 Department Group.xlsx', sheet_name = "Sheet1")
attrition_data_F002 = pd.merge(attrition_data_F002, F002_Department_Group, on='Masked Department', how='left')

attrition_data_F002  =attrition_data_F002[['Masked_Code'
                                            ,'YearQuarter'
                                            ,'AGE'
                                            ,'DATE_OF_JOINING'
                                            ,'Masked_Gender'
                                            ,'Masked Department'
                                            ,'Masked_Function'
                                            ,'PARTNER_NAME'
                                            ,'REPORTING_TO'
                                            ,'Masked_RM_ID'
                                            ,'Masked_Location'
                                            ,'EmpPerformanceRating'
                                            ,'LatestQtrProjectDays'
                                            ,'YAL'
                                            ,'QUARTER'
                                            ,'PP_SALARY'
                                            ,'LatestYrProjectDays'
                                            ,'LatestQtrProfLeadDevLearnHours'
                                            ,'Leave_Util_LatestQtrYr_Ratio'
                                            ,'PP_PERCENTILE_SALARY'
                                            ,'LatestQtr_snapshot_composite_score'
                                            ,'LatestQtrTechLearnHours'
                                            ,'LatestQtrProfLeadDevNumCourses'
                                            ,'LatestQtr_PROD_NONBILL_UTIL'
                                            ,'LatestYr_BILL_UTIL']] 

# print the features with missing values in log
check_missing_list_F002 = [col for col in attrition_data_F002.columns if attrition_data_F002[col].isnull().any()]
if len(check_missing_list_F002)> 0:
    log.write("F002 Features with missing values:\n")
    for i in check_missing_list_F002:
        log.write(i + "\n")
        print(i) 
else:
    log.write("F002 Features has no missing values:\n")

# check for missing values in F002 dataset
features_F002 =  ['Masked_Code'
                ,'YearQuarter'
                ,'AGE'
                ,'DATE_OF_JOINING'
                ,'Masked Department'
                ,'Masked_Function'
                ,'PARTNER_NAME'
                ,'REPORTING_TO'
                ,'Masked_RM_ID'
                ,'Masked_Location'
                ,'EmpPerformanceRating'
                ,'LatestQtrProjectDays'
                ,'YAL'
                ,'QUARTER'
                ,'PP_SALARY'
                ,'LatestYrProjectDays'
                ,'LatestQtrProfLeadDevLearnHours'
                ,'Leave_Util_LatestQtrYr_Ratio'
                ,'PP_PERCENTILE_SALARY'
                ,'LatestQtr_snapshot_composite_score'
                ,'LatestQtrTechLearnHours'
                ,'LatestQtrProfLeadDevNumCourses'
                ,'LatestQtr_PROD_NONBILL_UTIL'
                ,'LatestYr_BILL_UTIL']

check_missing_values_F002 = pd.DataFrame(attrition_data_F002[features_F002].isnull().sum())
check_missing_values_F002.columns = ['missing_count']
check_missing_values_F002 = check_missing_values_F002.sort_values(by=['missing_count'], ascending=False)
if ((check_missing_values_F002.iloc[0, 0]) >0):
    print("Data Quality Issue - Missing Values in F002 model variables")

os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Test\\')

# exporting data into excel for F002 model
writer = pd.ExcelWriter('Attrition_Analytics_F002.xlsx', engine='xlsxwriter', datetime_format='yyyy-mm-dd')
attrition_data_F002.to_excel(writer, index= False)
writer.save()

print("Successfully appended required variables and exported data file for F002 model\r\n")


### Data for F005 model

In [ ]:

#dividing data based on function
attrition_data_F005 = attrition_data[attrition_data['Masked_Function']== "F005"]

# get the department group for F005
attrition_data_F005['Masked_Department_Group'] = np.where((attrition_data_F005['Masked Department'] == "SL080"), 
                                                                    "SL052", 
                                                                            attrition_data_F005['Masked Department'])


attrition_data_F005 = attrition_data_F005[['Masked_Code'
                                            ,'YearQuarter'
                                            ,'AGE'
                                            ,'DATE_OF_JOINING'
                                            ,'Masked_Gender'
                                            ,'Masked Department'
                                            ,'Masked_Function'
                                            ,'PARTNER_NAME'                                             
                                            ,'REPORTING_TO'
                                            ,'Masked_RM_ID'
                                            ,'Masked_Location'
                                            ,'EmpPerformanceRating'
                                            ,'QUARTER'
                                            ,'PER_LatestQtrProjectDays'
                                            ,'YEARS_AT_COMPANY'
                                            ,'PP_SALARY'
                                            ,'PP_PERCENTILE_SALARY'
                                            ,'LatestYrProjectDays'
                                            ,'PER_LatestQtrProfLeadDevLearnHours'
                                            ,'PER_LatestQtrTechLearnHours'
                                            ,'LatestYr_snapshot_composite_score'
                                            ,'PER_LatestYrPerfSnapshot']]

# print the features with missing values in log
check_missing_list_F005 = [col for col in attrition_data_F005.columns if attrition_data_F005[col].isnull().any()]
if len(check_missing_list_F005)> 0:
    print("F005 Features with missing values:\n")
    for i in check_missing_list_F005:
        log.write(i + "\n")
        print(i) 
else:
    print("F005 Features has no missing values:\n")

# check for missing values in F005 dataset
features_F005 =  ['Masked_Code'
                ,'YearQuarter'
                ,'AGE'
                ,'DATE_OF_JOINING'
                ,'Masked Department'
                ,'Masked_Function'
                ,'PARTNER_NAME'                                             
                ,'REPORTING_TO'
                ,'Masked_RM_ID'
                ,'Masked_Location'
                ,'EmpPerformanceRating'
                ,'QUARTER'
                ,'PER_LatestQtrProjectDays'
                ,'YEARS_AT_COMPANY'
                ,'PP_SALARY'
                ,'PP_PERCENTILE_SALARY'
                ,'LatestYrProjectDays'
                ,'PER_LatestQtrProfLeadDevLearnHours'
                ,'PER_LatestQtrTechLearnHours'
                ,'LatestYr_snapshot_composite_score'
                ,'PER_LatestYrPerfSnapshot']

check_missing_values_F005 = pd.DataFrame(attrition_data_F005[features_F005].isnull().sum())
check_missing_values_F005.columns = ['missing_count']
check_missing_values_F005 = check_missing_values_F005.sort_values(by=['missing_count'], ascending=False)
if ((check_missing_values_F005.iloc[0, 0]) >0):
    print("Data Quality Issue - Missing Values in F005 model variables")
    log.write("Data Quality Issue - Missing Values in F005 model variables\r\n")
    log.close()
    sys.exit(1) 

# exporting data into excel for F005 model
writer = pd.ExcelWriter('Attrition_Analytics_F005.xlsx', engine='xlsxwriter', datetime_format='yyyy-mm-dd')
attrition_data_F005.to_excel(writer, index= False)
writer.save()

print("Successfully appended required variables and exported data file for F005 model\r\n")


In [ ]:
os.chdir('C:\\Users\\KNAGENDRA\\Desktop\\Attrition_model\\Data and Code/Code/Mapping Table/')

### Distribution Check for Model Features

In [ ]:
#Distribution Check for F002 Model Features
F002_distribution= pd.read_excel('Train_Distribution.xlsx', sheet_name = "F002")

F002_stats  = attrition_data_F002[['LatestQtrProjectDays'
                                    ,'YAL'
                                    ,'PP_SALARY'
                                    ,'LatestYrProjectDays'
                                    ,'LatestQtrProfLeadDevLearnHours'
                                    ,'Leave_Util_LatestQtrYr_Ratio'
                                    ,'PP_PERCENTILE_SALARY'
                                    ,'LatestQtr_snapshot_composite_score'
                                    ,'LatestQtrTechLearnHours'
                                    ,'LatestQtrProfLeadDevNumCourses'
                                    ,'LatestQtr_PROD_NONBILL_UTIL'
                                    ,'LatestYr_BILL_UTIL']] 


F002_stats = F002_stats.describe().loc[['mean','std']]
F002_stats = F002_stats.transpose()
F002_stats = F002_stats.reset_index()
F002_stats.columns = ['Feature', 'Scoring_Data_Mean', 'Scoring_Data_SD']
F002_stats = pd.merge(F002_distribution,F002_stats, on = 'Feature', how = 'left')

F002_stats['IsDistributionDiff'] = np.where(((F002_stats['Scoring_Data_Mean']/F002_stats['Train_Mean']) >= 0.75) &
                                              ((F002_stats['Scoring_Data_Mean']/F002_stats['Train_Mean']) <= 1.25), 0, 1)

if(sum(F002_stats['IsDistributionDiff'])/len(F002_stats) > 0.25):
    print("Different Distribution - F002 Model needs to be re-trained\r\n")


In [ ]:
#Distribution Check for F005 Model Features
F005_distribution= pd.read_excel('Train_Distribution.xlsx', sheet_name = "F005")

F005_stats  = attrition_data_F005[['PER_LatestQtrProjectDays'
                                    ,'YEARS_AT_COMPANY'
                                    ,'PP_SALARY'
                                    ,'PP_PERCENTILE_SALARY'
                                    ,'LatestYrProjectDays'
                                    ,'PER_LatestQtrProfLeadDevLearnHours'
                                    ,'PER_LatestQtrTechLearnHours'
                                    ,'LatestYr_snapshot_composite_score'
                                    ,'PER_LatestYrPerfSnapshot']] 

F005_stats = F005_stats.describe().loc[['mean','std']]
F005_stats = F005_stats.transpose()
F005_stats = F005_stats.reset_index()
F005_stats.columns = ['Feature', 'Scoring_Data_Mean', 'Scoring_Data_SD']
F005_stats = pd.merge(F005_distribution,F005_stats, on = 'Feature', how = 'left')

F005_stats['IsDistributionDiff'] = np.where(((F005_stats['Scoring_Data_Mean']/F005_stats['Train_Mean']) >= 0.75) &
                                              ((F005_stats['Scoring_Data_Mean']/F005_stats['Train_Mean']) <= 1.25), 0, 1)

if(sum(F005_stats['IsDistributionDiff'])/len(F005_stats) > 0.25):
    print("Different Distribution - F005 Model needs to be re-trained\r\n")
    log.write("Different Distribution - F005 Model needs to be re-trained\r\n")
else:
    print("Similar Distribution - F005 Model need not be re-trained\r\n")
    log.write("Similar Distribution - F005 Model need not be re-trained\r\n")

In [ ]:
log.write("Attrition Analytics ETL Executed\r\n")
log.write("\n")
log.write("\n")
log.close()